In [1]:
from typing import Dict, Optional
from datasets import Dataset, load_dataset, concatenate_datasets
import os
import glob
import json
import copy
import threading
import time
import random
from typing import Dict, Optional
import requests
import numpy as np
import re

import fasttext
# lang_detect = fasttext.load_model('../fastchat/modules/fasttext/lid.176.bin')

from fastchat.modules.answer_refiner import generate_refiner

import chromadb
from chromadb.config import Settings
import random
from fastchat.modules.embedder_adapter import Embedder, get_embedder
from fastchat.conversation import (
    SeparatorStyle,
)
from fastchat.model.model_adapter import get_conversation_template
import copy
from fastchat.train.data_modules.sft_dataset import load_sft_dataset, combine_dataset
from fastchat.train.data_modules.dpo_dataset import load_dpo_dataset
from fastchat.train.data_modules.dedup import (
    dedup_by_similarity,
    dedup_non_pair,
    dedup_repetition,
    dedup_math,
    dedup_too_much_token,
    dedup_short,
)
def random_select(dataset, num_select):
    return dataset.select(np.random.choice(list(range(len(dataset))), num_select))

/home/ados/anaconda3/envs/fastchat_2023dec/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# def dedup_by_similarity(dataset, prompt_template='chat-orca', target_text_len=100, n_results=100, distance_threshold = 0.6):
_dataset = train_dataset
prompt_template='vicuna'
target_text_len=100
n_results=100
distance_threshold = 0.35
    
if prompt_template == 'chat-orca':
    conv = get_conversation_template(prompt_template)
    system_message = conv.system_message
    sep_style = conv.sep_style
    sep = conv.sep
    prompt_user, prompt_bot = conv.roles

    len_sep_style = 0
    if sep_style == SeparatorStyle.ADD_COLON_TWO:
        len_sep_style = 1

    len_front = len(system_message) + len(sep) + len(prompt_user) + len_sep_style + 1
    len_rear = len(sep) + len(prompt_bot) + len_sep_style
    def filter_question(data):
        return { 
            # **data,
            'prompt': data['prompt'][len_front:-len_rear][:target_text_len]
        }

if prompt_template == 'vicuna':
    def filter_question(data):
        return {
            'prompt': data['conversations'][0]['value'][:target_text_len]
        }

question_dataset = _dataset.map(filter_question)

chroma_client = chromadb.Client(Settings(anonymized_telemetry=False))
embedder = get_embedder("ddobokki/klue-roberta-base-nli-sts-ko-en")
collection = chroma_client.create_collection(name="context", embedding_function=embedder.embed, metadata={"hnsw:space": "cosine"})
ids = []
# add
texts = question_dataset['prompt']
last_id = -1
new_ids = [f"id{i+last_id+1}" for i in range(len(texts))]
ids += new_ids
collection.add(documents=texts, ids=new_ids)

query_ids = copy.deepcopy(new_ids)
selected_ids = []
duplicated_ids = []

weird_ids = []
error_ids = []
while query_ids:
    current_id = random.choice(query_ids)
    if current_id in selected_ids:
        print("Warning: this is weird..")
        weird_ids.append(current_id)
        continue
    selected_ids.append(current_id)
    search_strings = [texts[int(current_id[2:])]]
    if collection.count() == 0:
        print("Warning: collection is empty. Forced break")
        break
    result = collection.query(query_texts=search_strings, n_results=min(n_results, len(query_ids)), include=['distances']) #'documents'

    search_ids = result['ids'][0]
    distances = result['distances'][0]
    remove_ids = []
    for idx in range(len(search_ids)):
        sid = search_ids[idx]
        dist = distances[idx]
        if dist < distance_threshold:
            remove_ids.append(sid)

    for rid in remove_ids:
        if rid in query_ids:
            query_ids.remove(rid)
            
    if remove_ids:
        duplicated_ids += remove_ids
        collection.delete(ids=remove_ids)
    else:
        print("Warning: this is error..")
        error_ids.append(current_id)

    print(f"Total:{len(new_ids)} Selected:{len(selected_ids)} current_dup:{len(remove_ids)} vector_store:{collection.count()} remained:{len(query_ids)} total_dup:{len(duplicated_ids)}", '\t\t\t\t\t', end='\r')

print('finished dedup data:', f"Total:{len(new_ids)} Selected:{len(selected_ids)} current_dup:{len(remove_ids)} vector_store:{collection.count()} remained:{len(query_ids)} total_dup:{len(duplicated_ids)}")

selected_ids = [int(sid[2:]) for sid in set(selected_ids)]

_dataset = _dataset.select(selected_ids)

# return dataset, selected_ids, query_ids

In [189]:
dataset = load_dpo_dataset(dpo_list5[2])

In [ ]:
dataset[2000]

In [190]:
mdpo.append(random_select(dataset, 1000))

In [ ]:
new_dataset_list = []
for d in dataset_list:
    new_dataset_list.append("\"" + d + "\"")
print(' '.join(new_dataset_list))

In [184]:
mdpo = []

In [133]:
msft = []

In [158]:
dataset_path = dataset_list[14]
print(dataset_path)
dataset = load_sft_dataset(dataset_path)
dataset

/data/llm_datasets/custom/deduped2/sharegpt_V3_format_translation(koen)-10000.json


Dataset({
    features: ['id', 'conversations', 'task'],
    num_rows: 10000
})

In [159]:
msft.append(random_select(dataset, 50))

In [191]:
combined_dataset = concatenate_datasets(mdpo)

In [161]:
combined_dataset = combined_dataset.shuffle(42)

In [ ]:
combined_dataset[2]

In [ ]:
# from fastchat.train.data_modules.sft_dataset import load_sft_dataset, combine_dataset
combined_dataset = concatenate_datasets([load_sft_dataset(dataset_path) for dataset_path in dataset_list])
combined_dataset.features

In [192]:
new_dataset = []
for data in combined_dataset:
    new_dataset.append(data)    


In [193]:
# combined_dataset.to_json("/data/llm_datasets/custom/ados_sft_v4.json")
with open("/data/llm_datasets/custom/ados/dpo/ados_mdpo_v2.json", "w") as json_file:
    json.dump(new_dataset, json_file)

In [194]:
dpo_dataset = ados_DPODataset("/data/llm_datasets/custom/ados/dpo/ados_mdpo_v2.json")

In [196]:
dpo_datamodule = dpo_dataset.make_dpo_data_module()

Extracting data files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1349.52it/s]
Generating train split: 11000 examples [00:01, 8384.35 examples/s]
Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11000/11000 [00:01<00:00, 9377.51 examples/s]


In [ ]:
from collections import defaultdict

"""
get and save language distributions
"""
def get_lang_distribution(dataset_list):
    lang_distribution = {}
    global_lang_dict = {'train': defaultdict(int), 'test': defaultdict(int)}
    for dataset_path in dataset_list:
        # dataset = load_sft_dataset(dataset_path, split=None)
        dataset = load_dpo_dataset(dataset_path, split=None)

        lang_splits = {}
        for split in list(dataset.keys()):
            print(f"{dataset_path}:{split}")
            _dataset = dataset[split]

            lang_dict = defaultdict(int)
            for data in _dataset:
                # conversations = data['conversations']
                conversations = data['chosen_response']
                langs = {}
                len_conv = 0
#                 for conv in conversations:
#                     # _from = conv['from']
#                     # _value = conv['value']
#                     _from = conv['role']
#                     _value = conv['content']

#                     len_conv += len(_value)
#                     lang, conf = lang_detect.predict(_value.replace('\n', ' '))
#                     lang = lang[0]
#                     if lang not in langs:
#                         langs[lang] = 1
#                     else:
#                         langs[lang] += 1
                _value = conversations

                len_conv += len(_value)
                lang, conf = lang_detect.predict(_value.replace('\n', ' '))
                lang = lang[0]
                if lang not in langs:
                    langs[lang] = 1
                else:
                    langs[lang] += 1

#                 if '__label__en' in langs:
#                     langs['__label__en'] -= 1

                if len(langs) == 0:
                    dominent_lang = "empty"
                else:
                    dominent_lang = max(langs)
                
                # if dominent_lang not in lang_dict:
                #     lang_dict[dominent_lang] = 1
                # else:
                global_lang_dict[split][dominent_lang] += 1
                lang_dict[dominent_lang] += 1
            lang_splits[split] = lang_dict
        lang_distribution[dataset_path] = lang_splits
    
    lang_distribution['total'] = global_lang_dict
    
    stat_dict = {'train': {}, 'test': {}}
    for split in ['train', 'test']:
        total_cnt = sum([value for value in global_lang_dict[split].values()])
        for key, value in global_lang_dict[split].items():
            stat_dict[split][key] = f"{value / total_cnt:.2%}"
    
    lang_distribution['stat'] = stat_dict
    
    return lang_distribution

lang_distribution = get_lang_distribution([dpo_list[2]])
lang_distribution

In [6]:
with open("/workspaces/data/llm_datasets/custom/lang_distribution_SFT_v4.json", "w") as json_file:
    json.dump(lang_distribution, json_file, indent=4)

In [ ]:
dataset = load_dataset("json", data_files="/workspaces/data/llm_datasets/custom/deduped/translated_sharegpt_V3_format_ko.json")
dataset

In [ ]:
dataset_koen = dataset['train'].select(range(10000, 15000))
dataset_enko = dataset['train'].select(range(15000, 20000))

In [ ]:
"""
SFT raw data and change format
"""

In [ ]:
new_dataset = []
for data in dataset_enko:
    conversations_ko = data['conversations']
    _id = data['id']
    
    conversations_en = lang_dict['__label__en'][_id]['conversations']
    
    for _idx in range(len(conversations_ko)):
        value_ko = conversations_ko[_idx]['value']
        value_en = conversations_en[_idx]['value']
        if len(value_ko) > 150: # 너무 짧은 데이터는 품질이 좋지 않아 제거
            new_dataset.append({
                'id': f"sharegpt_V3_format_{_id}_{_idx}",
                'task': 'enkotranslation',
                'conversations': [
                                    {'from': 'human', 'value': value_en},
                                    {'from': 'gpt', 'value': value_ko},
                                 ],
            })

In [ ]:
"""make lang_dict(sft)"""
dataset = load_dataset("json", data_files="/data/llm_datasets/custom/kodpo/untranslated/ultrafeedback_binarized.json")
enko_dataset = []
remained_dataset = []

lang_dict = {}
for data in dataset['train']:
    conversations = data['conversations']
    langs = {}
    len_conv = 0
    for conv in conversations:
        _from = conv['from']
        _value = conv['value']

        len_conv += len(_value)
        lang, conf = lang_detect.predict(_value.replace('\n', ' '))
        lang = lang[0]
        if lang not in langs:
            langs[lang] = 1
        else:
            langs[lang] += 1

    if '__label__en' in langs:
        langs['__label__en'] -= 1

    dominent_lang = max(langs)
    if dominent_lang not in lang_dict:
        lang_dict[dominent_lang] = [data]
    else:
        lang_dict[dominent_lang].append(data)
    # break

In [16]:
"""make lang_dict(dpo)"""
dataset = load_dataset("json", data_files="/data/llm_datasets/custom/kodpo/untranslated/truthy-dpo-v0.1.json")
enko_dataset = []
remained_dataset = []

lang_dict = {}
for data in dataset['train']:
    chosen = data['chosen']
    rejected = data['rejected']
    
    langs = {}

    lang_chosen = lang_detect.predict(chosen.replace('\n', ' '))[0][0]
    if lang_chosen not in langs:
        langs[lang_chosen] = 1
    else:
        langs[lang_chosen] += 1
        
    lang_rejected = lang_detect.predict(rejected.replace('\n', ' '))[0][0]
    if lang_rejected not in langs:
        langs[lang_rejected] = 1
    else:
        langs[lang_rejected] += 1

    if '__label__en' in langs:
        langs['__label__en'] -= 1

    dominent_lang = max(langs)
    if dominent_lang not in lang_dict:
        lang_dict[dominent_lang] = [data]
    else:
        lang_dict[dominent_lang].append(data)
    # break

Extracting data files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 902.97it/s]
Generating train split: 1016 examples [00:00, 21155.38 examples/s]


In [17]:
for key, value in lang_dict.items():
    print(key, len(value))

__label__en 1016


In [ ]:
import requests
api_server_url = "http://localhost:41002"
def count_total_token(conversations):
    num_token = 0
    for conv in conversations:
        input_json = {
            "model_name": "MingAI-70B-chat-orca_v0.42_2_dpo-GPTQ",
            "prompt": conv['value'],
        }

        ret = requests.post(api_server_url + "/count_token", json=input_json)

        output_json = ret.json()
        num_token += output_json['count']
    return num_token


for _idx in range(10):
    num_total_token = count_total_token(lang_dict['__label__en'][_idx])
    print(num_total_token)

In [ ]:
new_dataset += lang_dict['__label__pt'] + lang_dict['__label__es'] + lang_dict['__label__de'] + lang_dict['__label__zh'] + lang_dict['__label__fr']

In [ ]:
with open("/data/llm_datasets/custom/vicuna_format/lima_vicuna_format_ko.json", "w") as json_file:
    json.dump(new_dataset, json_file)

In [ ]:
new_dataset[2]

In [118]:
id_row = 'Ko_distilabel-math-preference-dpo_1'
filtered_row = dataset.filter(lambda sample: sample['id'] == id_row)
filtered_row[0]

Filter: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2418/2418 [00:00<00:00, 111092.18 examples/s]


{'input': 'What is the result of substituting `x = 5` and `y = -2` into the expression `2x^2 - 3y + 7`?.\n Take a deep breath, think step by step, and give an accurate response',
 'chosen': "To find the result of substituting `x = 5` and `y = -2` into the expression `2x^2 - 3y + 7`, we will substitute these values for the variables `x` and `y` in the given expression and simplify.\n\nSubstituting `x = 5` and `y = -2` into the expression, we get:\n\n2(5)^2 - 3(-2) + 7\n\nSimplifying, we have:\n\n2(25) + 6 + 7\n\nNow, let's perform the operations:\n\n50 + 6 + 7 = 63\n\nTherefore, the result of substituting `x = 5` and `y = -2` into the expression `2x^2 - 3y + 7` is 63.",
 'task': 'dpo',
 'id': 'Ko_distilabel-math-preference-dpo_1',
 'rejected': 'To find the result of substituting `x = 5` and `y = -2` into the expression `2x^2 - 3y + 7`, we need to replace `x` with `5` and `y` with `-2` in the expression and simplify it. \n\nSubstituting `x = 5` and `y = -2`:\n2(5)^2 - 3(-2) + 7\n\nSimpli

In [44]:
id_row = 'Ko_ultrafeedback_binarized_0649b7b08c0cf6f529901f80480b0ba7c7d29147638c39a9150ef15fd4e0ff9d'
filtered_row = temp_dataset.filter(lambda sample: sample['id'] == id_row)
filtered_row[0]

Filter: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46462/46462 [00:01<00:00, 40620.39 examples/s]


{'rejected': '최선을 다하겠습니다.',
 'id': 'Ko_ultrafeedback_binarized_0649b7b08c0cf6f529901f80480b0ba7c7d29147638c39a9150ef15fd4e0ff9d',
 'chosen': '간결한 답변을 드리기 위해 최선을 다하겠습니다.',
 'input': '내 질문에 10단어 이내로 답변하세요.',
 'task': 'dpo'}

In [ ]:
# en 먼저 필터링하고, 빈 데이터 채운 뒤에 합치기, 저장, 이후 dedup

In [94]:
# en 먼저 필터링하고
with open("/data/llm_datasets/custom/kodpo/temp_en.json", "w") as json_file:
    json.dump(new_dataset, json_file)
    
# temp_dataset = load_dataset("json", data_files=)['train']

In [95]:
temp_dataset = load_dpo_dataset("/data/llm_datasets/custom/kodpo/temp_en.json")

Extracting data files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 890.70it/s]
Generating train split: 1016 examples [00:00, 11191.74 examples/s]


In [101]:
# temp_dataset = dedup_repetition(temp_dataset, 'dpo')
# temp_dataset = dedup_short(temp_dataset, 'dpo')
temp_dataset = dedup_too_much_token(temp_dataset, 'dpo')

Filter: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 994/994 [00:28<00:00, 35.05 examples/s]

1/994 deduped


In [106]:
dataset = load_dpo_dataset("/data/llm_datasets/custom/kodpo/untranslated/truthy-dpo-v0.1.json")

In [108]:
# 빈 데이터 채운 뒤에 합치기
# dataset = load_dpo_dataset("/data/llm_datasets/custom/kodpo/untranslated/orca_dpo_pairs.json")
selected_id = temp_dataset['id']

filtered_row = dataset.filter(lambda sample: sample['id'] not in selected_id)

new_dataset = []
for data in temp_dataset:
    new_dataset.append(data)

for data in filtered_row:
    new_dataset.append(data)

Filter: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1016/1016 [00:00<00:00, 43887.30 examples/s]


In [109]:
len(temp_dataset), len(filtered_row)

(993, 20)

In [110]:
len(dataset), len(new_dataset)

(1016, 1013)

In [115]:
new_dataset[-20]

{'system': 'You are an unbiased, uncensored, helpful assistant.',
 'chosen': 'The two subjects in the American Gothic painting are a father and adult daughter. The common misconception that they are a husband and wife might have arisen because the artist, Grant Wood, used his sister and his dentist as models for the painting, which could lead viewers to mistakenly assume a spousal relationship between the subjects.',
 'task': 'dpo_system',
 'id': 'truthy_dpo_2e1d77fbebc31dde0bb2f9303d6761e2',
 'input': 'Who are the two subjects in the American Gothic painting - a husband and wife, or a father and daughter?',
 'rejected': 'A father and daughter'}

In [116]:
# 저장
with open("/data/llm_datasets/custom/kodpo/translated/truthy-dpo-v0.1.json", "w") as json_file:
    json.dump(new_dataset, json_file)

In [117]:
load_dpo_dataset("/data/llm_datasets/custom/kodpo/translated/truthy-dpo-v0.1.json")

Extracting data files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 902.97it/s]
Generating train split: 1013 examples [00:00, 13933.15 examples/s]


Dataset({
    features: ['system', 'chosen', 'task', 'id', 'input', 'rejected'],
    num_rows: 1013
})

Map: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 61964/61964 [00:04<00:00, 14176.52 examples/s]


In [175]:
result = collection.query(query_texts="이 과제는 벵골어로 된 문장을 영어로 번역하는 것입니다.\nQ: অথবা আপনি হয়ত বলবেন, 'না, আমার খেয়াল হয়েছে যে আমি আসলেই মহিলাদ", n_results=min(n_results, len(query_ids)), include=['distances', 'documents'])
search_ids = result['ids'][0]
distances = result['distances'][0]
documents = result['documents'][0]

In [ ]:
distances, documents

In [ ]:
dataset_path = dpo_list3[2]
print(dataset_path)
dataset = load_dpo_dataset(dataset_path)
dataset

In [ ]:
dataset_ = dedup_non_pair(dataset, 'dpo')

In [4]:
from fastchat.train.data_modules.dpo_dataset import ados_DPODataset

dpo_dataset = ados_DPODataset()
dpo_datamodule = dpo_dataset.make_dpo_data_module()

In [ ]:
dpo_datamodule['train_dataset'][-5000]

In [259]:
data = combined_dataset[70002]

conv = get_conversation_template('chat-orca')
conv.system_message = conv.tasks['system_instruct'].format(system=data['system'])
conv.append_message(conv.roles[0], data['input'])
conv.append_message(conv.roles[1], '')
prompt = conv.get_prompt()

conv.update_last_message(data['chosen'])
chosen = conv.get_prompt()[len(prompt):]
conv.update_last_message(data['rejected'])
rejected = conv.get_prompt()[len(prompt):]

In [26]:
from transformers import AutoTokenizer
# model_path = "/workspaces/disk0/data/llm_weights/Platypus2-70B-instruct/"
# model_path = "/workspaces/disk0/data/llm_weights/vicuna-13b-v1.5/"
model_path = "/data/llm_weights/custom_trained/M-DIE-M-10.7B_gpt4_dpo_ep2/"
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    model_max_length=16384,
    padding_side="right",
    use_fast=False,
)
tokenizer.pad_token = tokenizer.unk_token
from transformers.trainer_pt_utils import LabelSmoother
import torch
IGNORE_TOKEN_ID = LabelSmoother.ignore_index

In [109]:
#"/data/llm_datasets/custom/refined/sharegpt_V3_format_ko_selected_dedup2.json"
#"/data/llm_datasets/custom/vicuna_format/gpt_evol_1.3k-vicuna.json"
raw_data = load_sft_dataset("/data/llm_datasets/custom/refined/sharegpt_V3_format_ko_selected_dedup2.json")
raw_data = dedup_non_pair(raw_data)

Filter: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24094/24094 [00:01<00:00, 18172.54 examples/s]

31/24094 deduped


In [110]:
from fastchat.train.train import LazySupervisedDataset
dataset = LazySupervisedDataset(raw_data, tokenizer, data_format="chat-orca")

In [111]:
max_len = 16384

new_dataset = []

len_concat = 0
concat_input_ids = []
concat_labels = []
for data in dataset:
    input_ids = data['input_ids']
    labels = data['labels']
    attention_mask = data['attention_mask']

    full_input_ids = input_ids[attention_mask==True]
    full_labels = labels[attention_mask==True]

    len_concat += len(full_input_ids)
    if len_concat > max_len:
        new_input_ids = torch.cat(concat_input_ids)
        new_labels = torch.cat(concat_labels)
        assert len(new_input_ids) == len(new_labels)
        new_attention_mask = torch.ones(len(new_input_ids), dtype=torch.bool)
        
        new_input_ids = torch.cat((new_input_ids, torch.zeros(max_len - len(new_input_ids), dtype=input_ids.dtype)))
        new_labels = torch.cat((new_labels, torch.full((max_len - len(new_labels),), IGNORE_TOKEN_ID, dtype=labels.dtype)))
        new_attention_mask = torch.cat((new_attention_mask, torch.zeros(max_len - len(new_attention_mask), dtype=torch.bool)))
        
        new_dataset.append({
            'input_ids': new_input_ids.tolist(),
            'labels': new_labels.tolist(),
            'attention_mask': new_attention_mask.tolist(),
        })
        
        len_concat = len(full_input_ids)
        concat_input_ids = []
        concat_labels = []
        
    concat_input_ids.append(full_input_ids)
    concat_labels.append(full_labels)

In [112]:
len(new_dataset)

3758

In [113]:
data = new_dataset[0]

In [ ]:
tokenizer.decode(data['input_ids'])

In [ ]:
tokenizer.decode(torch.where(data['labels'] == IGNORE_TOKEN_ID, tokenizer.unk_token_id, data['labels']))

In [115]:
with open('/data/llm_datasets/custom/ados/yarn/ko-yarn-mistral-16k-sharegpt_V3_format_ko.json', "w") as json_file:
    json.dump(new_dataset, json_file)

In [118]:
def random_select(dataset, num_select):
    return dataset.select(np.random.choice(list(range(len(dataset))), num_select))
dataset = load_dpo_dataset("/data/llm_datasets/yarn-train-tokenized-16k-mistral/data/")
dataset = random_select(dataset, 200)
new_dataset = []
for _data in dataset:
    new_dataset.append(_data)
    
with open('/data/llm_datasets/custom/ados/yarn/yarn-train-tokenized-16k-mistral-200.json', "w") as json_file:
    json.dump(new_dataset, json_file)

Resolving data files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 89/89 [00:00<00:00, 95520.23it/s]


In [149]:
dataset1 = load_dpo_dataset("/data/llm_datasets/custom/ados/yarn/yarn-train-tokenized-16k-mistral-200.json")
dataset2 = load_dpo_dataset("/data/llm_datasets/custom/ados/yarn/ko-yarn-mistral-16k-gpt4_evol.json")
dataset3 = load_dpo_dataset("/data/llm_datasets/custom/ados/yarn/ko-yarn-mistral-16k-sharegpt_V3_format_ko.json")
dataset3 = random_select(dataset3, 1000 - len(dataset1) - len(dataset2))

In [150]:
from datasets import Features, Sequence, Value

dataset2 = dataset2.cast(
    Features({
        'attention_mask': Sequence(Value('int64')),
        'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
        'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
    })
)
dataset3 = dataset3.cast(
    Features({
        'attention_mask': Sequence(Value('int64')),
        'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
        'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
    })
)

Casting the dataset: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 699/699 [01:09<00:00, 10.09 examples/s]


In [151]:
conbined_dataset = concatenate_datasets((dataset1, dataset2, dataset3))

In [153]:
conbined_dataset = conbined_dataset.shuffle()

In [156]:
new_dataset = []
for _data in conbined_dataset:
    new_dataset.append(_data)
    
with open('/data/llm_datasets/custom/ados/yarn/ko-yarn-v1.json', "w") as json_file:
    json.dump(new_dataset, json_file)

In [ ]:
text = tokenizer.decode(data['input_ids'])
sentences = text.split('.')

len_split = 1024
splits = []
split = ""
for sentence in sentences:
    if len(split) < len_split:
        split += sentence + "."
    else:
        splits.append(split)
        split = ""

In [157]:
import re

def convert_to_seconds(time_string):
    # 시간 문자열을 분리하여 시, 분, 초를 추출한다.
    match = re.match(r'(\d+)시 (\d+)분 (\d+)초', time_string)
    if match is None:
        raise ValueError("잘못된 시간 형식입니다.")
    hour = int(match.group(1))
    minute = int(match.group(2))
    second = int(match.group(3))

    # 시간, 분, 초를 합산하여 총 초 단위 숫자를 계산한다.
    total_seconds = hour * 3600 + minute * 60 + second

    return total_seconds

# 예시 사용법
time_string = "03시 15분 30초"
seconds = convert_to_seconds(time_string)
print(f"{time_string}은 {seconds}초와 같습니다.")

03시 15분 30초은 11730초와 같습니다.


In [158]:
3600*3 + 15*60 + 30

11730

In [160]:
from lpr import LprYOLO

ModuleNotFoundError: No module named 'easyocr'

In [2]:
load_dpo_dataset("/data/llm_datasets/custom/merged/merged_korean_datasets-vicuna-v1.json")

Extracting data files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 472.86it/s]
Generating train split: 144874 examples [00:26, 5425.81 examples/s]


Dataset({
    features: ['conversations', 'id'],
    num_rows: 144874
})

In [3]:
dataset = load_sft_dataset("/data/llm_datasets/custom/ados/sft/ados_msft_v4.json")

In [9]:
dataset[1000]

{'task': None,
 'conversations': [{'from': 'human',
   'value': '잔센이라는 영웅에 대한 서사시를 써보세요. 그녀를 잔-잔-잔이라고 부르는 것도 괜찮습니다. 그녀는 독서, 스키, 운동을 좋아합니다. 그녀는 매일 아침 양파, 고추, 달걀, 의심스러운 고기를 섞은 이상한 혼합물을 먹습니다. 그녀는 영국, 텍사스, 애리조나에 살았어요. 지금은 유타에 살고 있습니다. 남편 바트는 거칠고 잘생겼어요.'},
  {'from': 'gpt',
   'value': '유타의 산이 높고 웅장한 곳에서\n공정한 소녀 잔센이 살았어요,\n책을 사랑하고 대지를 향한 열정을 가진,\n그리고 배가 부른 열정.\n그녀는 민첩하게 산을 스키를 탔고,\n그녀는 자신의 몸을 밀어붙였습니다,\n잔-잔-잔은 무시할 수없는 힘이있었습니다,\n진정한 두려움 없는 목격자.\n\n그녀는 매일 아침 이상한 아침으로 시작했습니다,\n양파, 고추, 달걀, 고기,\n하지만 잔센은 안전한 생활을하지 않았습니다,\n그녀는 독특한 비트를 가지고있었습니다.\n영국에서 태어나 텍사스와 애리조나에서 자랐습니다,\n잔센은 방랑벽이있었습니다,\n하지만 그녀는 거칠고 잘 생긴 바트라는 남자를 만났을 때,\n그녀의 마음은 그녀가 신뢰할 수있는 사람을 만났다는 것을 알았습니다.\n그래서 그들은 유타의 땅에 정착했습니다,\n그리고 함께 삶을 살았습니다,\n잔센의 힘과 아름다움은 모든 종류의 날씨와 함께 빛났습니다.\n그리고 세월이 지나도 잔센의 전설은 남아있을 것입니다,\n잔센은 대서사시의 여주인공이 될 것입니다.'},
  {'from': 'human', 'value': '아주 좋아요. 마치 제인 오스틴처럼 글을 쓸 수 있나요?'},
  {'from': 'gpt',
   'value': '유타의 구릉지대에서 살았어요,\n미스 잔센은 밝고 아름다운 여인이었어요,\n잔센이 그녀의 이름이었어요,\n진실하고 빛나는 빛.\n책과 운동을 사랑했고,\n몸과 마음을 조화롭게 유지했습니다,\n그리고

In [21]:
idx = 0
new_dataset = []

In [276]:
new_dataset.append({
    "task": "toc_extraction",
    "conversations": [
                        {'from': 'human', 'value': user_input},
                        {'from': 'gpt', 'value': output},
                     ],
    "id": f"toc_extraction_{idx}",
})
idx += 1
with open("/data/llm_datasets/custom/ados/sft/toc_extraction.json", "w") as json_file:
    json.dump(new_dataset, json_file)

In [275]:
user_input = """나경민의 주루사 때 카메라에 잡혔던 어느 롯데 팬의 절규가 큰 화제가 되었다.[14] 모래반지 빵야빵야처럼 파레이돌리아가 가능하다.

당시 나온 드립만 해도 가지각색이다.
- 야

이후 저 팬은 나경민에게 사인 유니폼을 받았다고 한다. 게다가 옷에 달고 있던 TWICE의 두 번째 투어 콘서트 'TWICELAND ZONE 2 : Fantasy Park'의 공식 굿즈인 캐릭터 핀 버튼[18]을 유니폼에 달고 있던 모습 때문에 ONCE들 사이에서도 유명세를 탔다.
"""
output = """없음
"""

In [224]:
len(new_dataset)

71

In [118]:
instruct = (
    "You are a Table of Contents extractor. "
    "User will speak to you questions. "
    "You must reply only with [목차(Table of Contents)] part extracted from the questions. "
    "You must keep original text. Do not change original text."
    "And you must not involve [dotted line, page number, 제목(title), content, explanation, summary, predicted]. "
    "do not write explanations."
)

In [119]:
instruct

'You are a Table of Contents extractor. User will speak to you questions. You must reply only with [목차(Table of Contents)] part extracted from the questions. You must keep original text. Do not change original text.And you must not involve [dotted line, page number, 제목(title), content, explanation, summary, predicted]. Do not write explanations.'

In [221]:
for data in new_dataset[-5:]:
    print(data['conversations'][1]['value'])

1. 개념
2. 형성 배경
3. 특징
4. 대표작가

1. 주제선정
2. 대한민국 신문 아카이브

3. 대한민국 신문 아카이브에 필요한 이용자서비스_다른 아카이브의 사례를 통하여

(2) 서평(Comment), 꼬리표(Tag) 기능

4. 결론



In [277]:
conv = get_conversation_template("chat-orca")

In [ ]:
ori_text = """목 차
I. 서 론 …………………………………………………………………………………………………………. 3
1. 실험 제목 …………………………………………………………………………………………….. 3
2. 실험 목적 …………………………………………………………………………………………….. 3
II. 본 론 …………………………………………………………………………………………………………. 3
1. 이론 및 원리 ……………………………………………………………………………………….. 4
2. 장치 및 방법 ……………………………………………………………………………………….. 4 
3. 실험 결과 …………………………………………………………………………………………….. 6
III. 결론 및 토의 …………………………………………………………………………………………… 10
IV. 참 고 문 헌 …………………………………………………………………………………………….. 12
"""
print(ori_text)

In [918]:
ori_text = """1 Introduction
In this paper, we present Mixtral 8x7B, a sparse mixture of experts model (SMoE) with open weights,
licensed under Apache 2.0. Mixtral outperforms Llama 2 70B and GPT-3.5 on most benchmarks. As
it only uses a subset of its parameters for every token, Mixtral allows faster inference speed at low
batch-sizes, and higher throughput at large batch-sizes.
Mixtral is a sparse mixture-of-experts network. It is a decoder-only model where the feedforward
block picks from a set of 8 distinct groups of parameters. At every layer, for every token, a router
network chooses two of these groups (the “experts”) to process the token and combine their output
additively. This technique increases the number of parameters of a model while controlling cost and
latency, as the model only uses a fraction of the total set of parameters per token.
Mixtral is pretrained with multilingual data using a context size of 32k tokens. It either matches
or exceeds the performance of Llama 2 70B and GPT-3.5, over several benchmarks. In particular,
arXiv:2401.04088v1 [cs.LG] 8 Jan 2024
Figure 1: Mixture of Experts Layer. Each input vector is assigned to 2 of the 8 experts by a router. The
layer’s output is the weighted sum of the outputs of the two selected experts. In Mixtral, an expert is a standard
feedforward block as in a vanilla transformer architecture.
Mixtral demonstrates superior capabilities in mathematics, code generation, and tasks that require
multilingual understanding, significantly outperforming Llama 2 70B in these domains. Experiments
show that Mixtral is able to successfully retrieve information from its context window of 32k tokens,
regardless of the sequence length and the location of the information in the sequence.
We also present Mixtral 8x7B – Instruct, a chat model fine-tuned to follow instructions using
supervised fine-tuning and Direct Preference Optimization [25]. Its performance notably surpasses
that of GPT-3.5 Turbo, Claude-2.1, Gemini Pro, and Llama 2 70B – chat model on human evaluation
benchmarks. Mixtral – Instruct also demonstrates reduced biases, and a more balanced sentiment
profile in benchmarks such as BBQ, and BOLD.
We release both Mixtral 8x7B and Mixtral 8x7B – Instruct under the Apache 2.0 license1
, free for
academic and commercial usage, ensuring broad accessibility and potential for diverse applications.
To enable the community to run Mixtral with a fully open-source stack, we submitted changes to
the vLLM project, which integrates Megablocks CUDA kernels for efficient inference. Skypilot also
allows the deployment of vLLM endpoints on any instance in the cloud.
2 Architectural details
Parameter Value
dim 4096
n_layers 32
head_dim 128
hidden_dim 14336
n_heads 32
n_kv_heads 8
context_len 32768
vocab_size 32000
num_experts 8
top_k_experts 2
Table 1: Model architecture.
Mixtral is based on a transformer architecture [31] and uses the same
modifications as described in [18], with the notable exceptions that Mixtral supports a fully dense context length of 32k tokens, and the feedforward blocks are replaced by Mixture-of-Expert layers (Section 2.1).
The model architecture parameters are summarized in Table 1.
2.1 Sparse Mixture of Experts
We present a brief overview of the Mixture of Experts layer (Figure 1).
For a more in-depth overview, see [12]. The output of the MoE module
for a given input x is determined by the weighted sum of the outputs
of the expert networks, where the weights are given by the gating
network’s output. i.e. given n expert networks {E0, Ei
, ..., En−1}, the
output of the expert layer is given by:
nX−1
i=0
G(x)i
· Ei(x).
Here, G(x)i denotes the n-dimensional output of the gating network for the i-th expert, and Ei(x)
is the output of the i-th expert network. If the gating vector is sparse, we can avoid computing
the outputs of experts whose gates are zero. There are multiple alternative ways of implementing
G(x) [6, 15, 35], but a simple and performant one is implemented by taking the softmax over the
Top-K logits of a linear layer [28]. We use
G(x) := Softmax(TopK(x · Wg)),
where (TopK(ℓ))i
:= ℓi
if ℓi
is among the top-K coordinates of logits ℓ ∈ R
n and (TopK(ℓ))i
:= −∞
otherwise. The value of K – the number of experts used per token – is a hyper-parameter that modulates the amount of compute used to process each token. If one increases n while keeping K fixed, one
1
https://mistral.ai/news/mixtral-of-experts/
2
can increase the model’s parameter count while keeping its computational cost effectively constant.
This motivates a distinction between the model’s total parameter count (commonly referenced as the
sparse parameter count), which grows with n, and the number of parameters used for processing an
individual token (called the active parameter count), which grows with K up to n.
MoE layers can be run efficiently on single GPUs with high performance specialized kernels. For
example, Megablocks [13] casts the feed-forward network (FFN) operations of the MoE layer as large
sparse matrix multiplications, significantly enhancing the execution speed and naturally handling
cases where different experts get a variable number of tokens assigned to them. Moreover, the
MoE layer can be distributed to multiple GPUs through standard Model Parallelism techniques, and
through a particular kind of partitioning strategy called Expert Parallelism (EP) [28]. During the MoE
layer’s execution, tokens meant to be processed by a specific expert are routed to the corresponding
GPU for processing, and the expert’s output is returned to the original token location. Note that EP
introduces challenges in load balancing, as it is essential to distribute the workload evenly across the
GPUs to prevent overloading individual GPUs or hitting computational bottlenecks.
In a Transformer model, the MoE layer is applied independently per token and replaces the
feed-forward (FFN) sub-block of the transformer block. For Mixtral we use the same SwiGLU
architecture as the expert function Ei(x) and set K = 2. This means each token is routed to two
SwiGLU sub-blocks with different sets of weights. Taking this all together, the output y for an input
token x is computed as:
y =
nX−1
i=0
Softmax(Top2(x · Wg))i
· SwiGLUi(x).
This formulation is similar to the GShard architecture [21], with the exceptions that we replace all
FFN sub-blocks by MoE layers while GShard replaces every other block, and that GShard uses a
more elaborate gating strategy for the second expert assigned to each token.
3 Results
We compare Mixtral to Llama, and re-run all benchmarks with our own evaluation pipeline for fair
comparison. We measure performance on a wide variety of tasks categorized as follow:
• Commonsense Reasoning (0-shot): Hellaswag [32], Winogrande [26], PIQA [3], SIQA [27],
OpenbookQA [22], ARC-Easy, ARC-Challenge [8], CommonsenseQA [30]
• World Knowledge (5-shot): NaturalQuestions [20], TriviaQA [19]
• Reading Comprehension (0-shot): BoolQ [7], QuAC [5]
• Math: GSM8K [9] (8-shot) with maj@8 and MATH [17] (4-shot) with maj@4
• Code: Humaneval [4] (0-shot) and MBPP [1] (3-shot)
• Popular aggregated results: MMLU [16] (5-shot), BBH [29] (3-shot), and AGI Eval [34]
(3-5-shot, English multiple-choice questions only)
Figure 2: Performance of Mixtral and different Llama models on a wide range of benchmarks. All models
were re-evaluated on all metrics with our evaluation pipeline for accurate comparison. Mixtral outperforms or
matches Llama 2 70B on all benchmarks. In particular, it is vastly superior in mathematics and code generation.
3
Model Active
Params MMLU HellaS WinoG PIQA Arc-e Arc-c NQ TriQA HumanE MBPP Math GSM8K
LLaMA 2 7B 7B 44.4% 77.1% 69.5% 77.9% 68.7% 43.2% 17.5% 56.6% 11.6% 26.1% 3.9% 16.0%
LLaMA 2 13B 13B 55.6% 80.7% 72.9% 80.8% 75.2% 48.8% 16.7% 64.0% 18.9% 35.4% 6.0% 34.3%
LLaMA 1 33B 33B 56.8% 83.7% 76.2% 82.2% 79.6% 54.4% 24.1% 68.5% 25.0% 40.9% 8.4% 44.1%
LLaMA 2 70B 70B 69.9% 85.4% 80.4% 82.6% 79.9% 56.5% 25.4% 73.0% 29.3% 49.8% 13.8% 69.6%
Mistral 7B 7B 62.5% 81.0% 74.2% 82.2% 80.5% 54.9% 23.2% 62.5% 26.2% 50.2% 12.7% 50.0%
Mixtral 8x7B 13B 70.6% 84.4% 77.2% 83.6% 83.1% 59.7% 30.6% 71.5% 40.2% 60.7% 28.4% 74.4%
Table 2: Comparison of Mixtral with Llama. Mixtral outperforms or matches Llama 2 70B performance on
almost all popular benchmarks while using 5x fewer active parameters during inference.
Figure 3: Results on MMLU, commonsense reasoning, world knowledge and reading comprehension,
math and code for Mistral (7B/8x7B) vs Llama 2 (7B/13B/70B). Mixtral largely outperforms Llama 2 70B
on all benchmarks, except on reading comprehension benchmarks while using 5x lower active parameters. It
is also vastly superior to Llama 2 70B on code and math.
Detailed results for Mixtral, Mistral 7B and Llama 2 7B/13B/70B and Llama 1 34B2
are reported
in Table 2. Figure 2 compares the performance of Mixtral with the Llama models in different
categories. Mixtral surpasses Llama 2 70B across most metrics. In particular, Mixtral displays a
superior performance in code and mathematics benchmarks.
Size and Efficiency. We compare our performance to the Llama 2 family, aiming to understand
Mixtral models’ efficiency in the cost-performance spectrum (see Figure 3). As a sparse Mixtureof-Experts model, Mixtral only uses 13B active parameters for each token. With 5x lower active
parameters, Mixtral is able to outperform Llama 2 70B across most categories.
Note that this analysis focuses on the active parameter count (see Section 2.1), which is directly
proportional to the inference compute cost, but does not consider the memory costs and hardware
utilization. The memory costs for serving Mixtral are proportional to its sparse parameter count,
47B, which is still smaller than Llama 2 70B. As for device utilization, we note that the SMoEs layer
introduces additional overhead due to the routing mechanism and due to the increased memory loads
when running more than one expert per device. They are more suitable for batched workloads where
one can reach a good degree of arithmetic intensity.
Comparison with Llama 2 70B and GPT-3.5. In Table 3, we report the performance of Mixtral 8x7B
compared to Llama 2 70B and GPT-3.5. We observe that Mixtral performs similarly or above the
two other models. On MMLU, Mixtral obtains a better performance, despite its significantly smaller
capacity (47B tokens compared to 70B). For MT Bench, we report the performance of the latest
GPT-3.5-Turbo model available, gpt-3.5-turbo-1106.
2
Since Llama 2 34B was not open-sourced, we report results for Llama 1 34B.
4
LLaMA 2 70B GPT-3.5 Mixtral 8x7B
MMLU
(MCQ in 57 subjects) 69.9% 70.0% 70.6%
HellaSwag
(10-shot) 87.1% 85.5% 86.7%
ARC Challenge
(25-shot) 85.1% 85.2% 85.8%
WinoGrande
(5-shot) 83.2% 81.6% 81.2%
MBPP
(pass@1) 49.8% 52.2% 60.7%
GSM-8K
(5-shot) 53.6% 57.1% 58.4%
MT Bench
(for Instruct Models) 6.86 8.32 8.30
Table 3: Comparison of Mixtral with Llama 2 70B and GPT-3.5. Mixtral outperforms or matches Llama 2
70B and GPT-3.5 performance on most metrics.
Evaluation Differences. On some benchmarks, there are some differences between our evaluation
protocol and the one reported in the Llama 2 paper: 1) on MBPP, we use the hand-verified subset 2)
on TriviaQA, we do not provide Wikipedia contexts.
3.1 Multilingual benchmarks
Compared to Mistral 7B, we significantly upsample the proportion of multilingual data during
pretraining. The extra capacity allows Mixtral to perform well on multilingual benchmarks while
maintaining a high accuracy in English. In particular, Mixtral significantly outperforms Llama 2 70B
in French, German, Spanish, and Italian, as shown in Table 4.
Active
Params
French German Spanish Italian
Model Arc-c HellaS MMLU Arc-c HellaS MMLU Arc-c HellaS MMLU Arc-c HellaS MMLU
LLaMA 1 33B 33B 39.3% 68.1% 49.9% 41.1% 63.3% 48.7% 45.7% 69.8% 52.3% 42.9% 65.4% 49.0%
LLaMA 2 70B 70B 49.9% 72.5% 64.3% 47.3% 68.7% 64.2% 50.5% 74.5% 66.0% 49.4% 70.9% 65.1%
Mixtral 8x7B 13B 58.2% 77.4% 70.9% 54.3% 73.0% 71.5% 55.4% 77.6% 72.5% 52.8% 75.1% 70.9%
Table 4: Comparison of Mixtral with Llama on Multilingual Benchmarks. On ARC Challenge, Hellaswag,
and MMLU, Mixtral outperforms Llama 2 70B on 4 languages: French, German, Spanish, and Italian.
3.2 Long range performance
To assess the capabilities of Mixtral to tackle long context, we evaluate it on the passkey retrieval
task introduced in [23], a synthetic task designed to measure the ability of the model to retrieve a
passkey inserted randomly in a long prompt. Results in Figure 4 (Left) show that Mixtral achieves a
100% retrieval accuracy regardless of the context length or the position of passkey in the sequence.
Figure 4 (Right) shows that the perplexity of Mixtral on a subset of the proof-pile dataset [2] decreases
monotonically as the size of the context increases.
Figure 4: Long range performance of Mixtral. (Left) Mixtral has 100% retrieval accuracy of the Passkey task
regardless of the location of the passkey and length of the input sequence. (Right) The perplexity of Mixtral on
the proof-pile dataset decreases monotonically as the context length increases.
5
3.3 Bias Benchmarks
Llama 2 70B Mixtral 8x7B
BBQ accuracy 51.5% 56.0%
BOLD sentiment score (avg ± std)
gender 0.293 ± 0.073 0.323 ±0.045
profession 0.218 ± 0.073 0.243 ± 0.087
religious_ideology 0.188 ± 0.133 0.144 ± 0.089
political_ideology 0.149 ± 0.140 0.186 ± 0.146
race 0.232 ± 0.049 0.232 ± 0.052
Figure 5: Bias Benchmarks. Compared Llama 2 70B,
Mixtral presents less bias (higher accuracy on BBQ, lower
std on BOLD) and displays more positive sentiment (higher
avg on BOLD).
To identify possible flaws to be corrected
by fine-tuning / preference modeling, we
measure the base model performance on
Bias Benchmark for QA (BBQ) [24] and
Bias in Open-Ended Language Generation
Dataset (BOLD) [10]. BBQ is a dataset
of hand-written question sets that target
attested social biases against nine different socially-relevant categories: age, disability status, gender identity, nationality,
physical appearance, race/ethnicity, religion,
socio-economic status, sexual orientation.
BOLD is a large-scale dataset that consists
of 23,679 English text generation prompts
for bias benchmarking across five domains.
We benchmark Llama 2 and Mixtral on BBQ and BOLD with our evaluation framework and report
the results in Table 5. Compared to Llama 2, Mixtral presents less bias on the BBQ benchmark
(56.0% vs 51.5%). For each group in BOLD, a higher average sentiment score means more positive
sentiments and a lower standard deviation indicates less bias within the group. Overall, Mixtral
displays more positive sentiments than Llama 2, with similar variances within each group.
4 Instruction Fine-tuning
We train Mixtral – Instruct using supervised fine-tuning (SFT) on an instruction dataset followed by
Direct Preference Optimization (DPO) [25] on a paired feedback dataset. Mixtral – Instruct reaches a
score of 8.30 on MT-Bench [33] (see Table 2), making it the best open-weights model as of December
2023. Independent human evaluation conducted by LMSys is reported in Figure 6
3
and shows that
Mixtral – Instruct outperforms GPT-3.5-Turbo, Gemini Pro, Claude-2.1, and Llama 2 70B chat.
Figure 6: LMSys Leaderboard. (Screenshot from Dec 22, 2023) Mixtral 8x7B Instruct v0.1 achieves an Arena
Elo rating of 1121 outperforming Claude-2.1 (1117), all versions of GPT-3.5-Turbo (1117 best), Gemini Pro
(1111), and Llama-2-70b-chat (1077). Mixtral is currently the best open-weights model by a large margin.
3
https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard
6
5 Routing analysis
In this section, we perform a small analysis on the expert selection by the router. In particular,
we are interested to see if during training some experts specialized to some specific domains (e.g.
mathematics, biology, philosophy, etc.).
To investigate this, we measure the distribution of selected experts on different subsets of The Pile
validation dataset [14]. Results are presented in Figure 7, for layers 0, 15, and 31 (layers 0 and 31
respectively being the first and the last layers of the model). Surprisingly, we do not observe obvious
patterns in the assignment of experts based on the topic. For instance, at all layers, the distribution of
expert assignment is very similar for ArXiv papers (written in Latex), for biology (PubMed Abstracts),
and for Philosophy (PhilPapers) documents.
Only for DM Mathematics we note a marginally different distribution of experts. This divergence is
likely a consequence of the dataset’s synthetic nature and its limited coverage of the natural language
spectrum, and is particularly noticeable at the first and last layers, where the hidden states are very
correlated to the input and output embeddings respectively.
This suggests that the router does exhibit some structured syntactic behavior. Figure 8 shows
examples of text from different domains (Python code, mathematics, and English), where each token
is highlighted with a background color corresponding to its selected expert. The figure shows that
words such as ‘self’ in Python and ‘Question’ in English often get routed through the same expert
even though they involve multiple tokens. Similarly, in code, the indentation tokens are always
assigned to the same experts, particularly at the first and last layers where the hidden states are more
correlated to the input and output of the model.
We also note from Figure 8 that consecutive tokens are often assigned the same experts. In fact, we
observe some degree of positional locality in The Pile datasets. Table 5 shows the proportion of consecutive tokens that get the same expert assignments per domain and layer. The proportion of repeated
0
0.05
0.10
0.15
0.20
layer: 0
0
0.05
0.10
0.15
0.20
layer: 15
0 1 2 3 4 5 6 7
0
0.05
0.10
0.15
0.20
layer: 31
Expert ID
Selection proportion
ArXiv
DM Mathematics
Github
Gutenberg
PhilPapers
PubMed Abstracts
StackExchange
Wikipedia (en)
Figure 7: Proportion of tokens assigned to each expert on different domains from The Pile dataset for
layers 0, 15, and 31. The gray dashed vertical line marks 1/8, i.e. the proportion expected with uniform
sampling. Here, we consider experts that are either selected as a first or second choice by the router. A
breakdown of the proportion of assignments done in each case cane be seen in Figure 9 in the Appendix.
7
First choice First or second choice
Layer 0 Layer 15 Layer 31 Layer 0 Layer 15 Layer 31
ArXiv 14.0% 27.9% 22.7% 46.5% 62.3% 52.9%
DM Mathematics 14.1% 28.4% 19.7% 44.9% 67.0% 44.5%
Github 14.9% 28.1% 19.7% 49.9% 66.9% 49.2%
Gutenberg 13.9% 26.1% 26.3% 49.5% 63.1% 52.2%
PhilPapers 13.6% 25.3% 22.1% 46.9% 61.9% 51.3%
PubMed Abstracts 14.2% 24.6% 22.0% 48.6% 61.6% 51.8%
StackExchange 13.6% 27.2% 23.6% 48.2% 64.6% 53.6%
Wikipedia (en) 14.4% 23.6% 25.3% 49.8% 62.1% 51.8%
Table 5: Percentage of expert assignment repetitions. We evaluate the proportion of times the same expert is
assigned to a token i and its following token i+1. We report whether the first chosen expert is the same, or whether
the same expert is observed as first or second choice in consecutive tokens. For reference, the expected proportion
of repetitions in the case of random assignments is 1
8 = 12.5% for “First choice” and 1 −
6
8
5
7 ≈ 46% for “First
and second choice”. Repetitions at the first layer are close to random, but are significantly higher at layers 15
and 31. The high number of repetitions shows that expert choice exhibits high temporal locality at these layers.
consecutive assignments is significantly higher than random for higher layers. This has implications
in how one might optimize the model for fast training and inference. For example, cases with high
locality are more likely to cause over-subscription of certain experts when doing Expert Parallelism.
Conversely, this locality can be leveraged for caching, as is done in [11]. A more complete view of
these same expert frequency is provided for all layers and across datasets in Figure 10 in the Appendix.
6 Conclusion
In this paper, we introduced Mixtral 8x7B, the first mixture-of-experts network to reach a state-of-theart performance among open-source models. Mixtral 8x7B Instruct outperforms Claude-2.1, Gemini Pro, and GPT-3.5 Turbo on human evaluation benchmarks. Because it only uses two experts at each
time step, Mixtral only uses 13B active parameters per token while outperforming the previous best
model using 70B parameters per token (Llama 2 70B). We are making our trained and fine-tuned models publicly available under the Apache 2.0 license. By sharing our models, we aim to facilitate the development of new techniques and applications that can benefit a wide range of industries and domains.
Figure 8: Text samples where each token is colored with the first expert choice. The selection of experts
appears to be more aligned with the syntax rather than the domain, especially at the initial and final layers.
8
Acknowledgements
We thank the CoreWeave and Scaleway teams for technical support as we trained our models. We
are grateful to NVIDIA for supporting us in integrating TensorRT-LLM and Triton and working
alongside us to make a sparse mixture of experts compatible with TensorRT-LLM.
"""
# print(ori_text)

In [895]:
ori_text = """1. 실험제목
등가속도운동 가속도 측정 실험

 

2. 실험목적
영상분석시스템을 이용하여 마찰이 없는 에어트랙에서 추에 매달린 물체의 운동을 통해 뉴턴의 운동 2법칙을 확인한다


3. 실험결과

 	1회	2회	3회
추(추걸이 포함)
질량(m₁) (g)	14.81	14.81	25.29
글라이더(연결자 포함)
질량(m₂) (g)	192.17	212.15	212.15
T-X그래프의 추세식(R²값 포함)	x = -32.365(t²) - 1.6224t + 122.68
R² = 1	x = -29.196(t²) - 1.1762t + 121.74
R² = 1	x = -48.747(t²) - 3.0515t + 120.9
R² = 1
가속도-측정치(a’) (cm/s²)	64.73	58.392	97.494
가속도-계산치(a) (cm/s²)	70.12	63.94	104.38
상대오차 |1-a’/a| * 100	7.686	8.676	6.597
 

4. 실험결과의 분석

일정한 가속도로 직선 운동하는 물체의 운동을 등가속도 운동이라 하며 뉴턴의 운동 2법칙을 이용하면 일정한 힘 ( 을 받는 질량 ( 인 물체의 가속도를 구할 수 있다.

 

일정 힘이 작용하는 경우 속도를 시간의 함수로 나타내면 v(t) = v0 + at 가 나오고 이 경우 질점의 위치 x 는 x(t) = x0 + v0t + (1/2)at² 이며 실험에서 얻은 질점의 추세식이 x(t) =C + Bt + At² 일 때 , 실험에 의한 질점의 가속도 a’ = 2A 를 구할 수 있었다 .


추와 추걸이의 무게가 m₁, 글라이더의 총 질량이 m₂ 일 때 (m₁ + m₂)a = m₁g 계산을 통해 가속도 a = m₁g / m₁ + m₂ 를 구할 수 있었다 .

 

실험결과에서 가속도 측정치는 가속도 계산치에 비해 더 적은 값을 나타냈는데 이는 글라이더의 마찰이나 공기저항 등으로 인한 결과로 보인다 . 또한 세 실험의 상대오차가 10미만으로 적은 값으로 나와 전체적으로 뉴턴의 제 2법칙이 잘 적용된 것으로 보인다.


실험2 에서는 실험 1 에서의 글라이더의 무게에 10g 추가하여 실험을 진행했다 . 글라이더의 질량이 증가하였으므로 마찰력이 증가하여 가속도가 실험 1 에 비해 줄어들었고 상대오차가 소폭 커진 것을 볼 수 있다.

 

실험3 에서는 추의 무게를 10g 추가하여 가속도를 증가시켰다 . 가속도 측정치와 계산치가 크게 증가하였고 상대오차는 실험 1, 2 와 비교하여 줄어든 것으로 보아 외부요인의 방해 없이 실험이 잘 진행되었음을 알 수 있다
"""

In [880]:
ori_text = """1 INTRODUCTION
Large language models (LLMs) (Radford et al., 2018; Devlin et al., 2018; Raffel et al., 2020; Brown
et al., 2020; OpenAI, 2023; Chowdhery et al., 2022; Anil et al., 2023; Thoppilan et al., 2022; Touvron
et al., 2023a;b) have revolutionized the field of artificial intelligence (AI) by providing a powerful
foundation for complex reasoning and problem-solving tasks. These models have the ability to
compress vast knowledge into neural networks, making them incredibly versatile agents. With a
chat interface, LLMs can perform tasks that were previously thought to be the exclusive domain of
humans, especially those involving creativity and expertise (OpenAI, 2022; Ouyang et al., 2022; Anil
et al., 2023; Google, 2023; Anthropic, 2023a;b). They can engage in natural language conversations
with humans, answering questions, providing information, and even generating creative content such
as stories, poems, and music. This has led to the development of a wide range of applications, from
chatbots and virtual assistants to language translation and summarization tools.
LLMs are not just limited to language tasks. They can also function as a generalist agent (Reed et al.,
2022; Bai et al., 2022a; Wang et al., 2023a; AutoGPT, 2023; Hong et al., 2023), collaborating with
external systems, tools, and models to achieve the objectives set by humans. For example, LLMs
can understand multimodal instructions (OpenAI, 2023; Bai et al., 2023; Liu et al., 2023a; Ye et al.,
2023; Dai et al., 2023; Peng et al., 2023b), execute code (Chen et al., 2021; Zheng et al., 2023; Li
et al., 2023d), use tools (Schick et al., 2023; LangChain, Inc., 2023; AutoGPT, 2023), and more.
This opens up a whole new world of possibilities for AI applications, from autonomous vehicles and
robotics to healthcare and finance. As these models continue to evolve and improve, we can expect
to see even more innovative and exciting applications in the years to come. Whether it’s helping us
solve complex problems, creating new forms of entertainment, or transforming the way we live and
work, LLMs are poised to play a central role in shaping the future of AI.
Pretrain
Models
RM
Models
SFT
Models
RLHF
Models
Qwen
Qwen-PMP Qwen-RM
Qwen-Chat Qwen-Chat-RLHF
Code-Qwen Code-Qwen-Chat
Math-Qwen-Chat
Qwen-VL Qwen-VL-Chat
Figure 1: Model Lineage of the Qwen Series. We have pretrained the language models, namely
QWEN, on massive datasets containing trillions of tokens. We then use SFT and RLHF to align
QWEN to human preference and thus we have QWEN-CHAT and specifically its improved version
QWEN-CHAT-RLHF. Additionally, we also develop specialized models for coding and mathematics,
such as CODE-QWEN, CODE-QWEN-CHAT, and MATH-QWEN-CHAT based on QWEN with similar
techniques. Note that we previously released the multimodal LLM, QWEN-VL and QWEN-VLCHAT (Bai et al., 2023), which are also based on our QWEN base models.
Despite their impressive capabilities, LLMs are often criticized for their lack of reproducibility,
steerability, and accessibility to service providers. In this work, we are pleased to present and release
the initial version of our LLM series, QWEN. QWEN is a moniker that derives from the Chinese phrase
Qianwen, which translates to “thousands of prompts” and conveys the notion of embracing a wide
range of inquiries. QWEN is a comprehensive language model series that encompasses distinct models
with varying parameter counts. The model series include the base pretrained language models, chat
models finetuned with human alignment techniques, i.e., supervised finetuning (SFT), reinforcement
learning with human feedback (RLHF), etc., as well as specialized models in coding and math. The
details are outlined below:
3
1. The base language models, namely QWEN, have undergone extensive training using up to 3
trillion tokens of diverse texts and codes, encompassing a wide range of areas. These models
have consistently demonstrated superior performance across a multitude of downstream
tasks, even when compared to their more significantly larger counterparts.
2. The QWEN-CHAT models have been carefully finetuned on a curated dataset relevant to task
performing, chat, tool use, agent, safety, etc. The benchmark evaluation demonstrates that
the SFT models can achieve superior performance. Furthermore, we have trained reward
models to mimic human preference and applied them in RLHF for chat models that can
produce responses preferred by humans. Through the human evaluation of a challenging test,
we find that QWEN-CHAT models trained with RLHF are highly competitive, still falling
behind GPT-4 on our benchmark.
3. In addition, we present specialized models called CODE-QWEN, which includes CODEQWEN-7B and CODE-QWEN-14B, as well as their chat models, CODE-QWEN-14BCHAT and CODE-QWEN-7B-CHAT. Specifically, CODE-QWEN has been pre-trained
on extensive datasets of code and further fine-tuned to handle conversations related to
code generation, debugging, and interpretation. The results of experiments conducted on
benchmark datasets, such as HumanEval (Chen et al., 2021), MBPP (Austin et al., 2021),
and HumanEvalPack (Muennighoff et al., 2023), demonstrate the high level of proficiency
of CODE-QWEN in code understanding and generation.
4. This research additionally introduces MATH-QWEN-CHAT specifically designed to tackle
mathematical problems. Our results show that both MATH-QWEN-7B-CHAT and MATHQWEN-14B-CHAT outperform open-sourced models in the same sizes with large margins
and are approaching GPT-3.5 on math-related benchmark datasets such as GSM8K (Cobbe
et al., 2021) and MATH (Hendrycks et al., 2021).
5. Besides, we have open-sourced QWEN-VL and QWEN-VL-CHAT, which have the versatile
ability to comprehend visual and language instructions. These models outperform the current
open-source vision-language models across various evaluation benchmarks and support text
recognition and visual grounding in both Chinese and English languages. Moreover, these
models enable multi-image conversations and storytelling. Further details can be found
in Bai et al. (2023).
Now, we officially open-source the 14B-parameter and 7B-parameter base pretrained models QWEN
and aligned chat models QWEN-CHAT2
. This release aims at providing more comprehensive and
powerful LLMs at developer- or application-friendly scales.
The structure of this report is as follows: Section 2 describes our approach to pretraining and results
of QWEN. Section 3 covers our methodology for alignment and reports the results of both automatic
evaluation and human evaluation. Additionally, this section describes details about our efforts in
building chat models capable of tool use, code interpreter, and agent. In Sections 4 and 5, we delve
into specialized models of coding and math and their performance. Section 6 provides an overview
of relevant related work, and Section 7 concludes this paper and points out our future work.
2 PRETRAINING
The pretraining stage involves learning vast amount of data to acquire a comprehensive understanding
of the world and its various complexities. This includes not only basic language capabilities but also
advanced skills such as arithmetic, coding, and logical reasoning. In this section, we introduce the
data, the model design and scaling, as well as the comprehensive evaluation results on benchmark
datasets.
2.1 DATA
The size of data has proven to be a crucial factor in developing a robust large language model,
as highlighted in previous research (Hoffmann et al., 2022; Touvron et al., 2023b). To create an
effective pretraining dataset, it is essential to ensure that the data are diverse and cover a wide range
2GitHub: https://github.com/QwenLM/Qwen.
4
Figure 2: Performance of GPT-4, GPT-3.5, the previous 13B SOTA, as well as QWEN-14B. We
demonstrate the results on 12 datasets covering multiple domains, including language understanding,
knowledge, reasoning, etc. QWEN significantly outperforms the previous SOTA of similar model
sizes, but still lag behind both GPT-3.5 and GPT-4.
of types, domains, and tasks. Our dataset is designed to meet these requirements and includes public
web documents, encyclopedia, books, codes, etc. Additionally, our dataset is multilingual, with a
significant portion of the data being in English and Chinese.
To ensure the quality of our pretraining data, we have developed a comprehensive data preprocessing
procedure. For public web data, we extract text from HTML and use language identification tools to
determine the language. To increase the diversity of our data, we employ deduplication techniques,
including exact-match deduplication after normalization and fuzzy deduplication using MinHash
and LSH algorithms. To filter out low-quality data, we employ a combination of rule-based and
machine-learning-based methods. Specifically, we use multiple models to score the content, including
language models, text-quality scoring models, and models for identifying potentially offensive or
inappropriate content. We also manually sample texts from various sources and review them to ensure
their quality. To further enhance the quality of our data, we selectively up-sample data from certain
sources, to ensure that our models are trained on a diverse range of high-quality content. In recent
studies (Zeng et al., 2022; Aribandi et al., 2021; Raffel et al., 2020), it has been demonstrated that
pretraining language models with multi-task instructions can enhance their zero-shot and few-shot
performance. To further enhance the performance of our model, we have incorporated high-quality
instruction data into our pretraining process. To safeguard the integrity of our benchmark assessment,
we have adopted a similar approach as Brown et al. (2020) and meticulously eliminated any instruction
5
th he ar ko vi zh ja tr id pl ru nl pt it de es fr en code
Languages
0.0
0.5
1.0
1.5
2.0
2.5
3.0
3.5
Compression Ratio
Model
LLaMA-7B
Baichuan-7B
ChatGLM2-6B
InternLM-7B
Qwen
Figure 3: Encoding compression rates of different models. We randomly selected 1 million
document corpora of each language to test and compare the encoding compression rates of different
models (with XLM-R (Conneau et al., 2019), which supports 100 languages, as the base value 1, not
shown in the figure). As can be seen, while ensuring the efficient decoding of Chinese, English, and
code, QWEN also achieves a high compression rate for many other languages (such as th, he, ar, ko,
vi, ja, tr, id, pl, ru, nl, pt, it, de, es, fr, etc.), equipping the model with strong scalability as well as
high training and inference efficiency in these languages.
samples that exhibit a 13-gram overlap with any data present in the test sets utilized in our evaluation.
Given the large number of downstream tasks, it is not feasible to repeat this filtering process for all
tasks. Instead, we have made sure that the instruction data for the reported tasks have undergone our
filtering process to ensure their accuracy and reliability. Finally, we have built a dataset of up to 3
trillion tokens.
2.2 TOKENIZATION
The design of vocabulary significantly impacts the training efficiency and the downstream task
performance. In this study, we utilize byte pair encoding (BPE) as our tokenization method, following
GPT-3.5 and GPT-4. We start with the open-source fast BPE tokenizer, tiktoken (Jain, 2022), and
select the vocabulary cl100k base as our starting point. To enhance the performance of our model on
multilingual downstream tasks, particularly in Chinese, we augment the vocabulary with commonly
used Chinese characters and words, as well as those in other languages. Also, following Touvron et al.
(2023a;b), we have split numbers into single digits. The final vocabulary size is approximately 152K.
The performance of the QWEN tokenizer in terms of compression is depicted in Figure 3. In this
comparison, we have evaluated QWEN against several other tokenizers, including XLM-R (Conneau
et al., 2019), LLaMA (Touvron et al., 2023a), Baichuan (Inc., 2023a), and InternLM (InternLM Team,
2023). Our findings reveal that QWEN achieves higher compression efficiency than its competitors in
most languages. This implies that the cost of serving can be significantly reduced since a smaller
number of tokens from QWEN can convey more information than its competitors. Furthermore, we
have conducted preliminary experiments to ensure that scaling the vocabulary size of QWEN does
not negatively impact the downstream performance of the pretrained model. Despite the increase
in vocabulary size, our experiments have shown that QWEN maintains its performance levels in
downstream evaluation.
2.3 ARCHITECTURE
QWEN is designed using a modified version of the Transformer architecture. Specifically, we have
adopted the recent open-source approach of training large language models, LLaMA (Touvron et al.,
2023a), which is widely regarded as the top open-source LLM. Our modifications to the architecture
include:
6
Table 1: Model sizes, architectures, and optimization hyper-parameters.
# of Params Hidden size Heads Layers Learning rate Batch size Training tokens
1.8B 2048 16 24 3.0 × 10−4
4M 2.2T
7B 4096 32 32 3.0 × 10−4
4M 2.4T
14B 5120 40 40 3.0 × 10−4
4M 3.0T
• Embedding and output projection. Based on preliminary experimental findings, we have
opted for the untied embedding approach instead of tying the weights of input embedding
and output projection. This decision was made in order to achieve better performance with
the price of memory costs.
• Positional embedding. We have chosen RoPE (Rotary Positional Embedding) (Su et al.,
2021) as our preferred option for incorporating positional information into our model. RoPE
has been widely adopted and has demonstrated success in contemporary large language
models, notably PaLM (Chowdhery et al., 2022; Anil et al., 2023) and LLaMA (Touvron
et al., 2023a;b). In particular, we have opted to use FP32 precision for the inverse frequency
matrix, rather than BF16 or FP16, in order to prioritize model performance and achieve
higher accuracy.
• Bias. For most layers, we remove biases following Chowdhery et al. (2022), but we add
biases in the QKV layer of attention to enhance the extrapolation ability of the model (Su,
2023b).
• Pre-Norm & RMSNorm. In modern Transformer models, pre-normalization is the most
widely used approach, which has been shown to improve training stability compared to
post-normalization. Recent research has suggested alternative methods for better training
stability, which we plan to explore in future versions of our model. Additionally, we have
replaced the traditional layer normalization technique described in (Ba et al., 2016) with
RMSNorm (Jiang et al., 2023). This change has resulted in equivalent performance while
also improving efficiency.
• Activation function. We have selected SwiGLU (Shazeer, 2020) as our activation function,
a combination of Swish (Ramachandran et al., 2017) and Gated Linear Unit (Dauphin et al.,
2017). Our initial experiments have shown that activation functions based on GLU generally
outperform other baseline options, such as GeLU (Hendrycks & Gimpel, 2016). As is
common practice in previous research, we have reduced the dimension of the feed-forward
network (FFN) from 4 times the hidden size to 8
3
of the hidden size.
2.4 TRAINING
To train QWEN, we follow the standard approach of autoregressive language modeling, as described
in Radford et al. (2018). This involves training the model to predict the next token based on the
context provided by the previous tokens. We train models with context lengths of 2048. To create
batches of data, we shuffle and merge the documents, and then truncate them to the specified context
lengths. To improve computational efficiency and reduce memory usage, we employ Flash Attention
in the attention modules (Dao et al., 2022). We adopt the standard optimizer AdamW (Kingma & Ba,
2014; Loshchilov & Hutter, 2017) for pretraining optimization. We set the hyperparameters β1 = 0.9,
β2 = 0.95, and ϵ = 10−8
. We use a cosine learning rate schedule with a specified peak learning rate
for each model size. The learning rate is decayed to a minimum learning rate of 10% of the peak
learning rate. All the models are trained with BFloat16 mixed precision for training stability.
2.5 CONTEXT LENGTH EXTENSION
Transformer models have a significant limitation in terms of the context length for their attention
mechanism. As the context length increases, the quadratic-complexity computation leads to a
drastic increase in both computation and memory costs. In this work, we have implemented simple
training-free techniques that are solely applied during inference to extend the context length of
the model. One of the key techniques we have used is NTK-aware interpolation (bloc97, 2023).
7
Table 2: Overall performance on widely-used benchmarks compared to open-source base models.
Our largest QWEN model with 14 billion parameters outperforms previous 13B SoTA models on all
datasets.
Model Params MMLU C-Eval GSM8K MATH HumanEval MBPP BBH
5-shot 5-shot 8-shot 4-shot 0-shot 3-shot 3-shot
MPT 7B 30.8 23.5 9.1 3.0 18.3 22.8 35.6
30B 47.9 - 15.2 3.1 25.0 32.8 38.0
Falcon 7B 27.8 - 6.8 2.3 - 11.2 28.0
40B 57.0 - 19.6 5.5 - 29.8 37.1
ChatGLM2 6B 47.9 51.7 32.4 6.5 - - 33.7
InternLM 7B 51.0 53.4 31.2 6.3 10.4 14.0 37.0
20B 62.1 58.8 52.6 7.9 25.6 35.6 52.5
Baichuan2 7B 54.7 56.3 24.6 5.6 18.3 24.2 41.6
13B 59.5 59.0 52.8 10.1 17.1 30.2 49.0
LLaMA
7B 35.6 27.3 11.0 2.9 12.8 17.7 33.5
13B 47.7 31.8 20.3 4.2 15.8 22.0 37.9
33B 58.7 37.5 42.3 7.1 21.7 30.2 50.0
65B 63.7 40.4 54.4 10.6 23.7 37.7 58.4
LLAMA 2
7B 46.8 32.5 16.7 3.3 12.8 20.8 38.2
13B 55.0 41.4 29.6 5.0 18.9 30.3 45.6
34B 62.6 - 42.2 6.2 22.6 33.0 44.1
70B 69.8 50.1 63.3 13.5 29.9 45.0 64.9
StableBeluga2 70B 68.6 51.4 69.6 14.6 28.0 11.4 69.3
QWEN
1.8B 44.6 54.7 21.2 5.6 17.1 14.8 28.2
7B 58.2 63.5 51.7 11.6 29.9 31.6 45.0
14B 66.3 72.1 61.3 24.8 32.3 40.8 53.4
Unlike position interpolation (PI) (Chen et al., 2023a) which scales each dimension of RoPE equally,
NTK-aware interpolation adjusts the base of RoPE to prevent the loss of high-frequency information
in a training-free manner. To further improve performance, we have also implemented a trivial
extension called dynamic NTK-aware interpolation, which is later formally discussed in (Peng et al.,
2023a). It dynamically changes the scale by chunks, avoiding severe performance degradation.
These techniques allow us to effectively extend the context length of Transformer models without
compromising their computational efficiency or accuracy.
QWEN additionally incorporates two attention mechanisms: LogN-Scaling (Chiang & Cholak, 2022;
Su, 2023a) and window attention (Beltagy et al., 2020). LogN-Scaling rescales the dot product of
the query and value by a factor that depends on the ratio of the context length to the training length,
ensuring that the entropy of the attention value remains stable as the context length grows. Window
attention restricts the attention to a limited context window, preventing the model from attending to
tokens that are too far away.
We also observed that the long-context modeling ability of our model varies across layers, with lower
layers being more sensitive in context length extension compared to the higher layers. To leverage
this observation, we assign different window sizes to each layer, using shorter windows for lower
layers and longer windows for higher layers.
2.6 EXPERIMENTAL RESULTS
To evaluate the zero-shot and few-shot learning capabilities of our models, we conduct a thorough benchmark assessment using a series of datasets. We compare QWEN with the most recent
open-source base models, including LLaMA (Touvron et al., 2023a), LLAMA 2 (Touvron et al.,
2023b), MPT (Mosaic ML, 2023), Falcon (Almazrouei et al., 2023), Baichuan2 (Yang et al., 2023),
ChatGLM2 (ChatGLM2 Team, 2023), InternLM (InternLM Team, 2023), XVERSE (Inc., 2023b),
and StableBeluga2 (Stability AI, 2023). Our evaluation covers a total of 7 popular benchmarks,
8
Table 3: Results of QWEN on long-context inference using various techniques. Our experimental
findings reveal that the application of our crucial techniques enables the model to consistently achieve
low perplexity as the context length increases. This suggests that these techniques play a significant
role in enhancing the model’s ability to comprehend and generate lengthy texts.
Model
Sequence Length
1024 2048 4096 8192 16384
QWEN-7B 4.23 3.78 39.35 469.81 2645.09
+ dynamic ntk 4.23 3.78 3.59 3.66 5.71
+ dynamic ntk + logn 4.23 3.78 3.58 3.56 4.62
+ dynamic ntk + logn + window attn 4.23 3.78 3.58 3.49 4.32
QWEN-14B - 3.46 22.79 334.65 3168.35
+ dynamic ntk + logn + window attn - 3.46 3.29 3.18 3.42
which are MMLU (5-shot) (Hendrycks et al., 2020), C-Eval (5-shot) (Huang et al., 2023), GSM8K
(8-shot) (Cobbe et al., 2021), MATH (4-shot) (Hendrycks et al., 2021), HumanEval (0-shot) (Chen
et al., 2021), MBPP (0-shot) (Austin et al., 2021), and BBH (Big Bench Hard) (3 shot) (Suzgun et al.,
2022). We aim to provide a comprehensive summary of the overall performance of our models across
these benchmarks.
In this evaluation, we focus on the base language models without alignment and collect the baselines’
best scores from their official results and OpenCompass (OpenCompass Team, 2023). The results are
presented in Table 2.
Our experimental results demonstrate that the three QWEN models exhibit exceptional performance
across all downstream tasks. It is worth noting that even the larger models, such as LLaMA2-70B, are
outperformed by QWEN-14B in 3 tasks. QWEN-7B also performs admirably, surpassing LLaMA2-
13B and achieving comparable results to Baichuan2-13B. Notably, despite having a relatively small
number of parameters, QWEN-1.8B is capable of competitive performance on certain tasks and even
outperforms larger models in some instances. The findings highlight the impressive capabilities of
the QWEN models, particularly QWEN-14B, and suggest that smaller models, such as QWEN-1.8B,
can still achieve strong performance in certain applications.
To evaluate the effectiveness of context length extension, Table 3 presents the test results on arXiv3
in
terms of perplexity (PPL). These results demonstrate that by combining NTK-aware interpolation,
LogN-Scaling, and layer-wise window assignment, we can effectively maintain the performance of
our models in the context of over 8192 tokens.
3 ALIGNMENT
Pretrained large language models have been found to be not aligned with human behavior, making
them unsuitable for serving as AI assistants in most cases. Recent research has shown that the use of
alignment techniques, such as supervised finetuning (SFT) and reinforcement learning from human
feedback (RLHF), can significantly improve the ability of language models to engage in natural
conversation. In this section, we will delve into the details of how QWEN models have been trained
using SFT and RLHF, and evaluate their performance in the context of chat-based assistance.
3.1 SUPERVISED FINETUNING
To gain an understanding of human behavior, the initial step is to carry out SFT, which finetunes a
pretrained LLM on chat-style data, including both queries and responses. In the following sections,
we will delve into the details of data construction and training methods.
3The dataset contains academic papers from https://arxiv.org.
9
3.1.1 DATA
To enhance the capabilities of our supervised finetuning datasets, we have annotated conversations
in multiple styles. While conventional datasets (Wei et al., 2022a) contain a vast amount of data
prompted with questions, instructions, and answers in natural language, our approach takes it a step
further by annotating human-style conversations. This practice, inspired by Ouyang et al. (2022),
aims at improving the model’s helpfulness by focusing on natural language generation for diverse
tasks. To ensure the model’s ability to generalize to a wide range of scenarios, we specifically
excluded data formatted in prompt templates that could potentially limit its capabilities. Furthermore,
we have prioritized the safety of the language model by annotating data related to safety concerns
such as violence, bias, and pornography.
In addition to data quality, we have observed that the training method can significantly impact the
final performance of the model. To achieve this, we utilized the ChatML-style format (OpenAI,
2022), which is a versatile meta language capable of describing both the metadata (such as roles)
and the content of a turn. This format enables the model to effectively distinguish between various
types of information, including system setup, user inputs, and assistant outputs, among others. By
leveraging this approach, we can enhance the model’s ability to accurately process and analyze
complex conversational data.
3.1.2 TRAINING
Consistent with pretraining, we also apply next-token prediction as the training task for SFT. We
apply the loss masks for the system and user inputs. More details are demonstrated in Section A.1.1.
The model’s training process utilizes the AdamW optimizer, with the following hyperparameters: β1
set to 0.9, β2 set to 0.95, and ϵ set to 10−8
. The sequence length is limited to 2048, and the batch
size is 128. The model undergoes a total of 4000 steps, with the learning rate gradually increased
over the first 1430 steps, reaching a peak of 2 × 10−6
. To prevent overfitting, weight decay is applied
with a value of 0.1, dropout is set to 0.1, and gradient clipping is enforced with a limit of 1.0.
3.2 REINFORCEMENT LEARNING FROM HUMAN FEEDBACK
While SFT has proven to be effective, we acknowledge that its generalization and creativity capabilities may be limited, and it is prone to overfitting. To address this issue, we have implemented
Reinforcement Learning from Human Feedback (RLHF) to further align SFT models with human
preferences, following the approaches of Ouyang et al. (2022); Christiano et al. (2017). This process
involves training a reward model and using Proximal Policy Optimization (PPO) (Schulman et al.,
2017) to conduct policy training.
3.2.1 REWARD MODEL
To create a successful reward model, like building a large language model (LLM), it is crucial to
first undergo pretraining and then finetuning. This pretraining process, also known as preference
model pretraining (PMP) (Bai et al., 2022b), necessitates a vast dataset of comparison data. This
dataset consists of sample pairs, each containing two distinct responses for a single query and their
corresponding preferences. Similarly, finetuning is also conducted on this type of comparison data,
but with a higher quality due to the presence of quality annotations.
During the fine-tuning phase, we gather a variety of prompts and adjust the reward model based on
human feedback for responses from the QWEN models. To ensure the diversity and complexity of
user prompts are properly taken into account, we have created a classification system with around
6600 detailed tags and implemented a balanced sampling algorithm that considers both diversity and
complexity when selecting prompts for annotation by the reward model (Lu et al., 2023). To generate
a wide range of responses, we have utilized QWEN models of different sizes and sampling strategies,
as diverse responses can help reduce annotation difficulties and enhance the performance of the
reward model. These responses are then evaluated by annotators following a standard annotation
guideline, and comparison pairs are formed based on their scores.
In creating the reward model, we utilize the same-sized pre-trained language model QWEN to initiate
the process. It is important to mention that we have incorporated a pooling layer into the original
10
Table 4: Test Accuracy of QWEN preference model pretraining (PMP) and reward model (RM) on
diverse human preference benchmark datasets.
Dataset QWEN QWEN Anthropic Anthropic OpenAI Stanford OpenAI
Helpful-base Helpful-online Helpful-base Helpful-online Summ. SHP PRM800K
PMP 62.68 61.62 76.52 65.43 69.60 60.05 70.59
RM 74.78 69.71 73.98 64.57 69.99 60.10 70.52
QWEN model to extract the reward for a sentence based on a specific end token. The learning rate for
this process has been set to a constant value of 3 × 10−6
, and the batch size is 64. Additionally, the
sequence length is set to 2048, and the training process lasts for a single epoch.
We adopted the accuracy on the test dataset as an important but not exclusive evaluation metric for
the reward model. In Table 4, we report the test pairwise accuracy of PMP and reward models on
diverse human preference benchmark datasets (Bai et al., 2022b; Stiennon et al., 2020; Ethayarajh
et al., 2022; Lightman et al., 2023). Specifically, QWEN Helpful-base and QWEN Helpful-online are
our proprietary datasets. The responses in QWEN Helpful-base are generated from QWEN without
RLHF, whereas QWEN Helpful-online includes responses from QWEN with RLHF. The results show
that the PMP model demonstrates high generalization capabilities on out-of-distribution data, and the
reward model demonstrates significant improvement on our QWEN reward datasets.
3.2.2 REINFORCEMENT LEARNING
Our Proximal Policy Optimization (PPO) process involves four models: the policy model, value
model, reference model, and reward model. Before starting the PPO procedure, we pause the policy
model’s updates and focus solely on updating the value model for 50 steps. This approach ensures
that the value model can adapt to different reward models effectively.
During the PPO operation, we use a strategy of sampling two responses for each query simultaneously.
This strategy has proven to be more effective based on our internal benchmarking evaluations. We set
the KL divergence coefficient to 0.04 and normalize the reward based on the running mean.
The policy and value models have learning rates of 1 × 10−6
and 5 × 10−6
, respectively. To enhance
training stability, we utilize value loss clipping with a clip value of 0.15. For inference, the policy
top-p is set to 0.9. Our findings indicate that although the entropy is slightly lower than when top-p is
set to 1.0, there is a faster increase in reward, ultimately resulting in consistently higher evaluation
rewards under similar conditions.
Additionally, we have implemented a pretrained gradient to mitigate the alignment tax. Empirical
findings indicate that, with this specific reward model, the KL penalty is adequately robust to
counteract the alignment tax in benchmarks that are not strictly code or math in nature, such as
those that test common sense knowledge and reading comprehension. It is imperative to utilize
a significantly larger volume of the pretrained data in comparison to the PPO data to ensure the
effectiveness of the pretrained gradient. Additionally, our empirical study suggests that an overly
large value for this coefficient can considerably impede the alignment to the reward model, eventually
compromising the ultimate alignment, while an overly small value would only have a marginal effect
on alignment tax reduction.
3.3 AUTOMATIC AND HUMAN EVALUATION OF ALIGNED MODELS
To showcase the effectiveness of our aligned models, we conduct a comparison with other aligned
models on well-established benchmarks, including MMLU (Hendrycks et al., 2020), C-Eval (Huang
et al., 2023), GSM8K (Cobbe et al., 2021), HumanEval (Chen et al., 2021), and BBH (Suzgun et al.,
2022). Besides the widely used few-shot setting, we test our aligned models in the zero-shot setting
to demonstrate how well the models follow instructions. The prompt in a zero-shot setting consists
of an instruction and a question without any previous examples in the context. The results of the
baselines are collected from their official reports and OpenCompass (OpenCompass Team, 2023).
The results in Table 5 demonstrate the effectiveness of our aligned models in understanding human
instructions and generating appropriate responses. QWEN-14B-Chat outperforms all other models
11
Table 5: Performance of aligned models on widely-used benchmarks. We report both zero-shot
and few-shot performance of the models.
Model Params MMLU C-Eval GSM8K HumanEval BBH
0-shot / 5-shot 0-shot / 5-shot 0-shot / 8-shot 0-shot 0-shot / 3-shot
Proprietary models
GPT-3.5 - - / 69.1 - / 52.5 - / 78.2 73.2 - / 70.1
GPT-4 - - / 83.0 - / 69.9 - / 91.4 86.6 - / 86.7
Open-source models
ChatGLM2 6B 45.5 / 46.0 50.1 / 52.6 - / 28.8 11.0 - / 32.7
InternLM-Chat 7B - / 51.1 - / 53.6 - / 33.0 14.6 - / 32.5
Baichuan2-Chat 7B - / 52.9 - / 55.6 - / 32.8 13.4 - / 35.8
13B - / 57.3 - / 56.7 - / 55.3 17.7 - / 49.9
LLAMA 2-CHAT
7B - / 46.2 - / 31.9 - / 26.3 12.2 - / 35.6
13B - / 54.6 - / 36.2 - / 37.1 18.9 - / 40.1
70B - / 63.8 - / 44.3 - / 59.3 32.3 - / 60.8
QWEN-CHAT
1.8B 42.4 / 43.9 50.7 / 50.3 27.8 / 19.5 14.6 27.1 / 25.0
7B 55.8 / 57.0 59.7 / 59.3 50.3 / 54.1 37.2 39.6 / 46.7
14B 64.6 / 66.5 69.8 / 71.7 60.1 / 59.3 43.9 46.9 / 58.7
except ChatGPT (OpenAI, 2022) and LLAMA 2-CHAT-70B (Touvron et al., 2023b) in all datasets,
including MMLU (Hendrycks et al., 2020), C-Eval (Huang et al., 2023), GSM8K (Cobbe et al., 2021),
HumanEval (Chen et al., 2021), and BBH (Suzgun et al., 2022). In particular, QWEN’s performance
in HumanEval, which measures the quality of generated codes, is significantly higher than that of
other open-source models.
Moreover, QWEN’s performance is consistently better than that of open-source models of similar size,
such as LLaMA2 (Touvron et al., 2023b), ChatGLM2 (ChatGLM2 Team, 2023), InternLM (InternLM
Team, 2023), and Baichuan2 (Yang et al., 2023). This suggests that our alignment approach, which
involves fine-tuning the model on a large dataset of human conversations, has been effective in
improving the model’s ability to understand and generate human-like language.
Despite this, we have reservations about the ability of traditional benchmark evaluation to accurately
measure the performance and potential of chat models trained with alignment techniques in today’s
landscape. The results mentioned earlier provide some evidence of our competitive standing, but we
believe that it is crucial to develop new evaluation methods specifically tailored to aligned models.
We believe that human evaluation is crucial, which is why we have created a carefully curated
dataset for this purpose. Our process involved collecting 300 instructions in Chinese that covered
a wide range of topics, including knowledge, language understanding, creative writing, coding,
and mathematics. To evaluate the performance of different models, we chose the SFT version of
QWEN-CHAT-7B and the SFT and RLHF versions of QWEN-CHAT-14B, and added two strong
baselines, GPT-3.5 and GPT-44
, for comparison. For each instruction, we asked three annotators to
rank the model responses by the overall score of helpfulness, informativeness, validity, and other
relevant factors. Our dataset and evaluation methodology provides a comprehensive and rigorous
assessment of the capabilities of different language models in various domains.
Figure 4 illustrates the win rates of the various models. For each model, we report the percentage of
wins, ties, and losses against GPT-3.5, with the segments of each bar from bottom to top representing
these statistics. The experimental results clearly demonstrate that the RLHF model outperforms
the SFT models by significant margins, indicating that RLHF can encourage the model to generate
responses that are more preferred by humans. In terms of overall performance, we find that the
RLHF model significantly outperforms the SFT models, falling behind GPT-4. This indicates
the effectiveness of RLHF for aligning to human preference. To provide a more comprehensive
understanding of the models’ performance, we include a case study with examples from different
models in Appendix A.2.2. Nonetheless, it remains difficult to accurately capture the gap between our
4To obtain the results from the models, we use the OpenAI APIs of GPT-3.5-turbo-0613 and GPT-4-0613.
12
Average Knowledge Language UnderstandingCreative Writing Math Coding 0
20
40
60
80
100
28.4
31.1
24.0
35.3
23.3
17.5
28.3
31.1
46.0
19.7
29.2
22.5
43.3
37.8
30.0
45.0
47.5
60.0
30.7
32.8
29.3
34.7
34.2
15.8
32.3
33.3
54.7
22.7
34.2
25.0
37.0
33.9
16.0
42.7
31.7
59.2
38.7
42.2
39.3
39.7
39.2
30.0
28.9
32.8
33.3
21.0
39.2
26.7
32.4
25.0
27.3
39.3
21.7
43.3
43.8
37.2
36.0
49.7
45.8
46.7
32.2
40.0
47.3
19.0
33.3
33.3
24.0
22.8
16.7
31.3
20.8
20.0
Winrate (v.s. GPT-3.5)
Qwen-7B-Chat (SFT) Qwen-14B-Chat (SFT) Qwen-14B-Chat (RLHF) GPT-4
Figure 4: Results of the human evaluation for chat models. We compare Qwen-7B (SFT), Qwen14B (SFT), Qwen-14B (RLHF), as well as GPT-4 against GPT-3.5. Each bar segment represents the
percentage of wins, ties, and losses, from bottom to top. On average, the RLHF model outperforms
the SFT model. The dataset consists of 300 Chinese instructions.
models and the proprietary models. As such, a more extensive and rigorous assessment is required
for the chat models.
3.4 TOOL USE, CODE INTERPRETER, AND AGENT
Table 6: Performance of QWEN on the in-house Chinese benchmark that evaluates its ability to use
unseen tools via ReAct prompting.
Model Params Tool Selection (Acc.↑) Tool Input (Rouge-L↑) False Positive Error (%)↓
GPT-4 - 95 90 15.0
GPT-3.5 - 85 88 75.0
QWEN-CHAT
1.8B 92 89 19.3
7B 98 91 7.3
14B 98 93 2.4
The QWEN models, which are designed to be versatile, have the remarkable ability to assist with
(semi-)automating daily tasks by leveraging their skills in tool-use and planning. As such, they can
serve as agents or copilots to help streamline various tasks. We explore QWEN’s proficiency in the
following areas:
• Utilizing unseen tools through ReAct prompting (Yao et al., 2022) (see Table 6).
• Using a Python code interpreter to enhance math reasoning, data analysis, and more (see
Table 7 and Table 8).
• Functioning as an agent that accesses Hugging Face’s extensive collection of multimodal
models while engaging with humans (see Table 9).
13
Table 7: The proportion of code generated by QWEN that is executable on the in-house evaluation
benchmark for Code Interpreter. This benchmark examines QWEN’s coding proficiency in math
problem solving, data visualization, and general purposes. CODE LLAMA underperforms on
visualization tasks because it hallucinates non-existent columns solely based on CSV file names (see
Figure 5).
Model Params
Category
Math (%) Visualization (%) General (%) All (%)
GPT-4 - 91.9 85.9 82.8 86.8
GPT-3.5 - 89.2 65.0 74.1 72.9
LLAMA 2-CHAT
7B 41.9 33.1 24.1 33.6
13B 50.0 40.5 48.3 44.4
CODE LLAMA-INSTRUCT
7B 85.1 54.0 70.7 65.1
13B 93.2 55.8 74.1 68.8
InternLM-Chat 7B v1.1 78.4 44.2 62.1 56.3
20B 70.3 44.2 65.5 54.9
QWEN-CHAT
1.8B 33.8 30.1 8.6 26.8
7B 82.4 64.4 67.2 70.2
14B 89.2 84.1 65.5 81.7
Table 8: Correctness of the final response on the in-house evaluation benchmark for Code Interpreter.
Visualization-Hard tasks involve planning multiple steps, while Visualization-Easy tasks do not.
Visualization-All measures both types of tasks. CODE LLAMA excels in performing VisualizationEasy tasks but tends to underperform in Visualization-Hard tasks, due to its inclination to hallucinate
non-existent columns based on the name of a CSV file (see Figure 5).
Model Params
Category
Math (%) Vis.-Hard (%) Vis.-Easy (%) Vis.-All (%)
GPT-4 - 82.8 66.7 60.8 63.8
GPT-3.5 - 47.3 33.3 55.7 44.2
LLAMA 2-CHAT
7B 3.9 14.3 39.2 26.4
13B 8.3 8.3 40.5 23.9
CODE LLAMA-INSTRUCT
7B 14.3 26.2 60.8 42.9
13B 28.2 27.4 62.0 44.2
InternLM-Chat 7B v1.1 28.5 4.8 40.5 22.1
20B 34.6 21.4 45.6 33.1
QWEN-CHAT
1.8B 14.7 3.6 20.3 11.7
7B 41.9 40.5 54.4 47.2
14B 58.4 53.6 59.5 56.4
14
Table 9: Results of QWEN-Chat on the Hugging Face Agent benchmark.
Task Model Params
Metric
Tool Selection ↑ Tool Used ↑ Code Correctness ↑
Run Mode
GPT-4 - 100 100 97.4
GPT-3.5 - 95.4 96.3 87.0
Starcoder-Base 15B 86.1 87.0 68.9
Starcoder 15B 87.0 88.0 68.9
QWEN-CHAT
1.8B 85.2 84.3 61.1
7B 87.0 87.0 71.5
14B 93.5 94.4 87.0
Chat Mode
GPT-4 - 97.9 97.9 98.5
GPT-3.5 - 97.3 96.8 89.6
Starcoder-Base 15B 97.9 97.9 91.1
Starcoder 15B 97.9 97.9 89.6
QWEN-CHAT
1.8B 93.6 93.6 73.2
7B 94.7 94.7 85.1
14B 97.9 97.9 95.5
To enhance QWEN’s capabilities as an agent or copilot, we employ the self-instruct (Wang et al.,
2023c) strategy for SFT. Specifically, we utilize the in-context learning capability of QWEN for
self-instruction. By providing a few examples, we can prompt QWEN to generate more relevant
queries and generate outputs that follow a specific format, such as ReAct (Yao et al., 2022). We then
apply rules and involve human annotators to filter out any noisy samples. Afterwards, the samples
are incorporated into QWEN’s training data, resulting in an updated version of QWEN that is more
dependable for self-instruction. We iterate through this process multiple times until we gather an
ample number of samples that possess both exceptional quality and a wide range of diversity. As a
result, our final collection consists of around 2000 high-quality samples.
During the finetuning process, we mix these high-quality samples with all the other general-purpose
SFT samples, rather than introducing an additional training stage. By doing so, we are able to retain
essential general-purpose capabilities that are also pertinent for constructing agent applications.
Using Tools via ReAct Prompting We have created and made publicly available a benchmark
for evaluating QWEN’s ability to call plugins, tools, functions, or APIs using ReAct Prompting (see
Qwen Team, Alibaba Group, 2023b). To ensure fair evaluation, we have excluded any plugins that
were included in QWEN’s training set from the evaluation set. The benchmark assesses the model’s
accuracy in selecting the correct plugin from a pool of up to five candidates, as well as the plausibility
of the parameters passed into the plugin and the frequency of false positives. In this evaluation, a
false positive occurs when the model incorrectly invokes a plugin in response to a query, despite not
being required to do so.
The results presented in Table 6 demonstrate that QWEN consistently achieves higher accuracy in
identifying the relevance of a query to the available tools as the model size increases. However,
the table also highlights that beyond a certain point, there is little improvement in performance
when it comes to selecting the appropriate tool and providing relevant arguments. This suggests that
the current preliminary benchmark may be relatively easy and may require further enhancement in
future iterations. It is worth noting that GPT-3.5 stands out as an exception, displaying suboptimal
performance on this particular benchmark. This could potentially be attributed to the fact that the
benchmark primarily focuses on the Chinese language, which may not align well with GPT-3.5’s
capabilities. Additionally, we observe that GPT-3.5 tends to attempt to use at least one tool, even if
the query cannot be effectively addressed by the provided tools.
Using Code Interpreter for Math Reasoning and Data Analysis The Python code interpreter
is widely regarded as a powerful tool for augmenting the capabilities of an LLM agent. It is
15
worth investigating whether QWEN can harness the full potential of this interpreter to enhance its
performance in diverse domains, such as mathematical reasoning and data analysis. To facilitate this
exploration, we have developed and made publicly available a benchmark that is specifically tailored
for this purpose (see Qwen Team, Alibaba Group, 2023a).
The benchmark encompasses three primary categories of tasks: math problem-solving, data visualization, and other general-purpose tasks like file post-processing and web crawling. Within the
visualization tasks, we differentiate between two levels of difficulty. The easier level can be achieved
by simply writing and executing a single code snippet without the need for advanced planning skills.
However, the more challenging level requires strategic planning and executing multiple code snippets
in a sequential manner. This is because the subsequent code must be written based on the output of
the previous code. For example, an agent may need to examine the structure of a CSV file using one
code snippet before proceeding to write and execute additional code to create a plot.
Regarding evaluation metrics, we consider both the executability and correctness of the generated
code. To elaborate on the correctness metrics, for math problems, we measure accuracy by verifying
if the ground truth numerical answer is present in both the code execution result and the final response.
When it comes to data visualization, we assess accuracy by utilizing QWEN-VL (Bai et al., 2023),
a powerful multimodal language model. QWEN-VL is capable of answering text questions paired
with images, and we rely on it to confirm whether the image generated by the code fulfills the user’s
request.
The results regarding executability and correctness are presented in Table 7 and Table 8, respectively.
It is evident that CODE LLAMA generally outperforms LLAMA 2, its generalist counterpart, which
is not surprising since this benchmark specifically requires coding skills. However, it is worth noting
that specialist models that are optimized for code synthesis do not necessarily outperform generalist
models. This is due to the fact that this benchmark encompasses various skills beyond coding,
such as abstracting math problems into equations, understanding language-specified constraints, and
responding in the specified format such as ReAct. Notably, QWEN-7B-CHAT and QWEN-14B-CHAT
surpass all other open-source alternatives of similar scale significantly, despite being generalist
models.
Serving as a Hugging Face Agent Hugging Face provides a framework called the Hugging Face
Agent or Transformers Agent (Hugging Face, 2023), which empowers LLM agents with a curated set
of multimodal tools, including speech recognition and image synthesis. This framework allows an
LLM agent to interact with humans, interpret natural language commands, and employ the provided
tools as needed.
To evaluate QWEN’s effectiveness as a Hugging Face agent, we utilized the evaluation benchmarks
offered by Hugging Face. The results are presented in Table 9. The evaluation results reveal that
QWEN performs quite well in comparison to other open-source alternatives, only slightly behind the
proprietary GPT-4, demonstrating QWEN’s competitive capabilities.
4 CODE-QWEN: SPECIALIZED MODEL FOR CODING
Training on domain-specific data has been shown to be highly effective, particularly in the case
of code pretraining and finetuning. A language model that has been reinforced with training on
code data can serve as a valuable tool for coding, debugging, and interpretation, among other tasks.
In this work, we have developed a series of generalist models using pretraining and alignment
techniques. Building on this foundation, we have created domain-specific models for coding by
leveraging the base language models of QWEN, including continued pretrained model, CODE-QWEN
and supervised finetuned model, CODE-QWEN-CHAT. Both models have 14 billion and 7 billion
parameters versions.
4.1 CODE PRETRAINING
We believe that relying solely on code data for pretraining can result in a significant loss of the ability
to function as a versatile assistant. Unlike previous approaches that focused solely on pretraining
on code data (Li et al., 2022; 2023d), we take a different approach (Roziere et al., 2023) by starting `
with our base models QWEN trained on a combination of text and code data, and then continuing to
16
pretrain on the code data. We continue to pretrain the models on a total of around 90 billion tokens.
During the pre-training phase, we initialize the model using the base language models QWEN. Many
applications that rely on specialized models for coding may encounter lengthy contextual scenarios,
such as tool usage and code interpretation, as mentioned in Section 3.4. To address this issue, we
train our models with context lengths of up to 8192. Similar to base model training in Section
2.4, we employ Flash Attention (Dao et al., 2022) in the attention modules, and adopt the standard
optimizer AdamW (Kingma & Ba, 2014; Loshchilov & Hutter, 2017), setting β1 = 0.9, β2 = 0.95,
and ϵ = 10−8
. We set the learning rate as 6.0 × 10−5
for CODE-QWEN-14B and 3.0 × 10−5
for
CODE-QWEN-7B, with 3% warm up iterations and no learning rate decays.
4.2 CODE SUPERVISED FINE-TUNING
After conducting a series of empirical experiments, we have determined that the multi-stage SFT
strategy yields the best performance compared to other methods. In the supervised fine-tuning stage,
the model CODE-QWEN-CHAT initialized by the code foundation model CODE-QWEN are optimized
by the AdamW (Kingma & Ba, 2014; Loshchilov & Hutter, 2017) optimizer (β1 = 0.9, β2 = 0.95,
ϵ = 10−8
) with a learning rate of 2.0 × 10−6
and 1.0 × 10−5
for the 14B and 7B model respectively.
The learning rate increases to the peaking value with the cosine learning rate schedule (3% warm-up
steps) and then remains constant.
4.3 EVALUATION
Our CODE-QWEN models have been compared with both proprietary and open-source language
models, as shown in Tables 10 and 11. These tables present the results of our evaluation on the
test sets of Humaneval (Chen et al., 2021), MBPP (Austin et al., 2021), and the multi-lingual code
generation benchmark HUMANEVALPACK (Muennighoff et al., 2023). The comparison is based on
the pass@1 performance of the models on these benchmark datasets. The results of this comparison
are clearly demonstrated in Tables 10 and 11.
Our analysis reveals that specialized models, specifically CODE-QWEN and CODE-QWEN-CHAT, significantly outperform previous baselines with similar parameter counts, such as OCTOGEEX (Muennighoff et al., 2023), InstructCodeT5+ (Wang et al., 2023d), and CodeGeeX2 (Zheng et al., 2023). In
fact, these models even rival the performance of larger models like Starcoder (Li et al., 2023d).
When compared to some of the extremely large-scale closed-source models, CODE-QWEN and CODEQWEN-CHAT demonstrate clear advantages in terms of pass@1. However, it is important to note that
these models fall behind the state-of-the-art methods, such as GPT-4, in general. Nonetheless, with
the continued scaling of both model size and data size, we believe that this gap can be narrowed in
the near future.
It is crucial to emphasize that the evaluations mentioned previously are insufficient for grasping
the full extent of the strengths and weaknesses of the models. In our opinion, it is necessary to
develop more rigorous tests to enable us to accurately assess our relative performance in comparison
to GPT-4.
5 MATH-QWEN: SPECIALIZED MODEL FOR MATHEMATICS REASONING
We have created a mathematics-specialized model series called MATH-QWEN-CHAT, which is
built on top of the QWEN pretrained language models. Specifically, we have developed assistant
models that are specifically designed to excel in arithmetic and mathematics and are aligned with
human behavior. We are releasing two versions of this model series, MATH-QWEN-14B-CHAT and
MATH-QWEN-7B-CHAT, which have 14 billion and 7 billion parameters, respectively.
5.1 TRAINING
We carry out math SFT on our augmented math instructional dataset for mathematics reasoning,
and therefore we obtain the chat model, MATH-QWEN-CHAT, directly. Owing to shorter average
lengths of the math SFT data, we use a sequence length of 1024 for faster training. Most user inputs
in the math SFT dataset are examination questions, and it is easy for the model to predict the input
17
Table 10: Results of pass@1 (%) on HumanEval and MBPP. Most scores are retrieved from the
papers of StarCoder (Li et al., 2023d), CodeT5+ (Wang et al., 2023d), WizardCoder (Luo et al.,
2023b) and CODE LLAMA (Roziere et al., 2023). `
Model Params HumanEval MBPP
Proprietary models
PaLM 540B 26.2 36.8
PaLM-Coder 540B 36.0 47.0
PaLM 2-S - 37.6 50.0
Code-Cushman-001 - 33.5 45.9
Code-Davinci-002 - 47.0 58.1
GPT-3.5 - 73.2 -
GPT-4 - 86.6 -
Open-source models
LLAMA 2
7B 12.2 20.8
13B 20.1 27.6
34B 22.6 33.8
70B 30.5 45.4
CodeGen-Multi 16B 18.3 20.9
CodeGen-Mono 16B 29.3 35.3
CodeGeeX2 6B 35.9 -
StarCoder-Prompted 15B 40.8 49.5
CodeT5+ 16B 30.9 -
InstructCodeT5+ 16B 35.0 -
CODE LLAMA
7B 33.5 41.4
13B 36.0 47.0
34B 48.8 55.0
CODE LLAMA-INSTRUCT
7B 34.8 44.4
13B 42.7 49.4
34B 41.5 57.0
CODE LLAMA-PYTHON
7B 38.4 47.6
13B 43.3 49.0
34B 53.7 56.2
UNNATURAL CODE LLAMA 34B 62.2 61.2
WizardCoder-Python 13B 64.0 55.6
34B 73.2 61.2
QWEN-CHAT
7B 37.2 35.8
14B 43.9 46.4
CODE-QWEN
7B 40.2 41.8
14B 45.1 51.4
CODE-QWEN-CHAT
7B 43.3 44.2
14B 66.4 52.4
18
Table 11: Zero-shot pass@1 (%) performance on the HUMANEVALPACK (synthesize) benchmark. The baseline results are partly from OCTOPACK (Muennighoff et al., 2023).
Model Params
Programming Language
Python JavaScript Java Go C++ Rust Avg.
Proprietary models
GPT-4 - 86.6 82.9 81.7 72.6 78.7 67.1 78.3
Open-source models
InstructCodeT5+ 16B 37.0 18.9 17.4 9.5 19.8 0.3 17.1
StarChat-β 15B 33.5 31.4 26.7 25.5 26.6 14.0 26.3
StarCoder 15B 33.6 30.8 30.2 17.6 31.6 21.8 27.6
CodeGeeX2 6B 35.9 32.2 30.8 22.5 29.3 18.1 28.1
OCTOGEEX 6B 44.7 33.8 36.9 21.9 32.3 15.7 30.9
OCTOCODER 15B 46.2 39.2 38.2 30.4 35.6 23.4 35.5
WizardCoder 15B 59.8 49.5 36.1 36.4 40.9 20.2 40.5
QWEN-CHAT
7B 37.2 23.2 32.9 20.7 22.0 9.1 24.2
14B 43.9 38.4 42.7 34.1 24.4 18.9 33.7
CODE-QWEN
7B 40.2 40.4 40.2 26.2 20.7 15.8 30.6
14B 45.1 51.8 57.3 39.6 18.2 20.7 38.8
CODE-QWEN-CHAT
7B 43.3 41.5 49.4 29.3 32.9 20.1 36.1
14B 66.4 58.5 56.1 47.6 54.2 28.7 51.9
Table 12: Results of models on mathematical reasoning. We report the accuracy of QWEN for all
benchmarks using greedy decoding. For MATH, we are reporting QWEN’s performances on the test
set from Lightman et al. (2023).
Model Params GSM8K MATH Math401 Math23K
Proprietary models
GPT-4 - 92.0 42.5 83.5 74.0
GPT-3.5 - 80.8 34.1 75.1 60.0
Minerva
8B 16.2 14.1 - -
62B 52.4 27.6 - -
540B 58.8 33.6 - -
Open-source models
LLaMA-1 RFT 7B 46.5 5.2 - -
13B 52.1 5.1 - -
WizardMath
7B 54.9 10.7 - -
13B 63.9 14.0 - -
70B 81.6 22.7 - -
GAIRMath-Abel
7B 59.7 13.0 - -
13B 66.4 17.3 - -
70B 83.6 28.3 - -
QWEN-CHAT
7B 50.3 6.8 57.4 51.2
14B 60.1 18.4 70.1 67.0
MATH-QWEN-CHAT
7B 62.5 17.2 80.8 75.4
14B 69.8 24.2 85.0 78.4
19
format and it is meaningless for the model to predict the input condition and numbers which could be
random. Thus, we mask the inputs of the system and user to avoid loss computation on them and find
masking them accelerates the convergence during our preliminary experiments. For optimization, we
use the AdamW optimizer with the same hyperparameters of SFT except that we use a peak learning
rate of 2 × 10−5
and a training step of 50 000.
5.2 EVALUATION
We evaluate models on the test sets of GSM8K (Grade school math) (Cobbe et al., 2021), MATH
(Challenging competition math problems) (Hendrycks et al., 2021), Math401 (Arithmetic ability) (Yuan et al., 2023b), and Math23K (Chinese grade school math) (Wang et al., 2017). We compare
MATH-QWEN-CHAT with proprietary models ChatGPT and Minerva (Lewkowycz et al., 2022) and
open-sourced math-specialized model RFT (Yuan et al., 2023a), WizardMath (Luo et al., 2023a), and
GAIRMath-Abel (Chern et al., 2023a) in Table 12. MATH-QWEN-CHAT models show better math
reasoning and arithmetic abilities compared to open-sourced models and QWEN-CHAT models of
similar sizes. Compared to proprietary models, MATH-QWEN-7B-CHAT outperforms Minerva-8B in
MATH. MATH-QWEN-14B-CHAT is chasing Minerva-62B and GPT-3.5 in GSM8K and MATH and
delivers better performance on arithmetic ability and Chinese math problems.
6 RELATED WORK
6.1 LARGE LANGUAGE MODELS
The excitement of LLM began with the introduction of the Transformer architecture (Vaswani et al.,
2017), which was then applied to pretraining large-scale data by researchers such as Radford et al.
(2018); Devlin et al. (2018); Liu et al. (2019). These efforts led to significant success in transfer
learning, with model sizes growing from 100 million to over 10 billion parameters (Raffel et al.,
2020; Shoeybi et al., 2019).
In 2020, the release of GPT-3, a massive language model that is 10 times larger than T5, demonstrated
the incredible potential of few-shot and zero-shot learning through prompt engineering and in-context
learning, and later chain-of-thought prompting (Wei et al., 2022c). This success has led to a number
of studies exploring the possibilities of further scaling these models (Scao et al., 2022; Zhang et al.,
2022; Du et al., 2021; Zeng et al., 2022; Lepikhin et al., 2020; Fedus et al., 2022; Du et al., 2022;
Black et al., 2022; Rae et al., 2021; Hoffmann et al., 2022; Chowdhery et al., 2022; Thoppilan
et al., 2022). As a result, the community has come to view these large language models as essential
foundations for downstream models (Bommasani et al., 2021).
The birth of ChatGPT (OpenAI, 2022) and the subsequent launch of GPT-4 (OpenAI, 2023) marked
two historic moments in the field of artificial intelligence, demonstrating that large language models
(LLMs) can serve as effective AI assistants capable of communicating with humans. These events
have sparked interests among researchers and developers in building language models that are aligned
with human values and potentially even capable of achieving artificial general intelligence (AGI) (Anil
et al., 2023; Anthropic, 2023a;b).
One notable development in this area is the emergence of open-source LLMs, specifically
LLaMA (Touvron et al., 2023a) and LLAMA 2 (Touvron et al., 2023b), which have been recognized
as the most powerful open-source language models ever created. This has led to a surge of activity
in the open-source community (Wolf et al., 2019), with a series of large language models being
developed collaboratively to build upon this progress (Mosaic ML, 2023; Almazrouei et al., 2023;
ChatGLM2 Team, 2023; Yang et al., 2023; InternLM Team, 2023).
6.2 ALIGNMENT
The community was impressed by the surprising effectiveness of alignment on LLMs. Previously,
LLMs without alignment often struggle with issues such as repetitive generation, hallucination,
and deviation from human preferences. Since 2021, researchers have been diligently working on
developing methods to enhance the performance of LLMs in downstream tasks (Wei et al., 2022a;
Sanh et al., 2021; Longpre et al., 2023; Chung et al., 2022; Muennighoff et al., 2022). Furthermore,
20
researchers have been actively exploring ways to align LLMs with human instructions (Ouyang et al.,
2022; Askell et al., 2021; Bai et al., 2022b;c). One major challenge in alignment research is the
difficulty of collecting data. While OpenAI has utilized its platform to gather human prompts or
instructions, it is not feasible for others to collect such data.
However, there has been some progress in this area, such as the self-instruct approach proposed
in Wang et al. (2023c). This innovative work offers a potential solution to the data collection problem
in alignment research. As a result, there has been a surge in open-source chat data, including
Alpaca (Taori et al., 2023), MOSS (Sun et al., 2023a), Dolly (Conover et al., 2023), Evol-Instruct (Xu
et al., 2023b), and others (Sun et al., 2023b; Xu et al., 2023a;c; Chen et al., 2023c; Ding et al.,
2023; Ji et al., 2023; Yang, 2023). Similarly, there has been an increase in open-source chat models,
such as Alpaca (Taori et al., 2023), Vicuna (Chiang et al., 2023), Guanaco (Dettmers et al., 2023),
MOSS (Sun et al., 2023a), WizardLM (Xu et al., 2023b), and others (Xu et al., 2023c; Chen et al.,
2023c; Ding et al., 2023; Wang et al., 2023b).
To train an effective chat model, available solutions are mostly based on SFT and RLHF (Ouyang
et al., 2022). While SFT is similar to pretraining, it focuses on instruction following using the
aforementioned data. However, for many developers, the limited memory capacity is a major obstacle
to further research in SFT. As a result, parameter-efficient tuning methods, such as LoRA (Hu et al.,
2021) and Q-LoRA (Dettmers et al., 2023), have gained popularity in the community. LoRA tunes
only low-rank adapters, while Q-LoRA builds on LoRA and utilizes 4-bit quantized LLMs and
paged attention (Dettmers et al., 2022; Frantar et al., 2022; Kwon et al., 2023). In terms of RLHF,
recent methods such as PPO (Schulman et al., 2017; Touvron et al., 2023b) have been adopted, but
there are also alternative techniques aimed at addressing the complexity of optimization, such as
RRHF (Yuan et al., 2023c), DPO (Rafailov et al., 2023), and PRO (Song et al., 2023). Despite the
ongoing debate about the effectiveness of RLHF, more evidence is needed to understand how it
enhances the intelligence of LLMs and what potential drawbacks it may have.
6.3 TOOL USE AND AGENTS
LLM’s planning function allows for the invocation of tools, such as APIs or agent capabilities,
through in-context learning, as demonstrated by Schick et al. (2023). Yao et al. (2022) introduced
ReAct, a generation format that enables the model to generate thoughts on which tool to use, accept
input from API observations, and generate a response. GPT-3.5 and GPT-4, when prompted with
few shots, have shown consistent and impressive performance. In addition to tool usage, LLMs can
utilize external memory sources like knowledge bases (Hu et al., 2023; Zhong et al., 2023b) or search
engines (Nakano et al., 2021; Liu et al., 2023b) to generate more accurate and informative answers.
This has led to the popularity of frameworks like LangChain (LangChain, Inc., 2023). The research on
LLMs for tool use has also sparked interest in building agents with LLM capabilities, such as agents
that can call different AI models (Shen et al., 2023; Li et al., 2023a), embodied lifelong learning or
multimodal agents (Wang et al., 2023a; Driess et al., 2023), and multiple agents interacting with each
other and even building a micro-society (Chen et al., 2023b; Li et al., 2023b; Xu et al., 2023d; Hong
et al., 2023).
6.4 LLM FOR CODING
Previous research has demonstrated that LLMs possess remarkable capabilities in code understanding
and generation, particularly those with massive numbers of parameters (Chowdhery et al., 2022;
Anil et al., 2023; Rae et al., 2021; Hoffmann et al., 2022). Moreover, several LLMs have been pretrained, continued pre-trained, or fine-tuned on coding-related data, which has resulted in significantly
improved performance compared to general-purpose LLMs. These models include Codex Chen
et al. (2021), AlphaCode (Li et al., 2022), SantaCoder (Allal et al., 2023), Starcoder-Base (Li et al.,
2023d), InCoder (Fried et al., 2022), CodeT5 (Wang et al., 2021), CodeGeeX (Zheng et al., 2023),
and CODE LLAMA (Roziere et al., 2023). In addition to these models, recent studies have focused on `
developing specialized alignment techniques for coding, such as Code Llama-Instruct (Roziere et al., `
2023) and StarCoder (Li et al., 2023d). These models can assist developers in various code-related
tasks, including code generation (Chen et al., 2021; Austin et al., 2021), code completion (Zhang
et al., 2023a), code translation (Szafraniec et al., 2023), bug fixing (Muennighoff et al., 2023), code
refinement (Liu et al., 2023c), and code question answering (Liu & Wan, 2021). In a word, LLMs
21
have the potential to revolutionize the field of coding by providing developers with powerful tools for
code comprehension, generation, and related tasks.
6.5 LLM FOR MATHEMATICS
LLMs with a certain model scale have been found to possess the ability to perform mathematical
reasoning (Wei et al., 2022b; Suzgun et al., 2022). In order to encourage LLMs to achieve better
performance on math-related tasks, researchers have employed techniques such as chain-of-thought
prompting (Wei et al., 2022c) and scratchpad (Nye et al., 2021), which have shown promising results.
Additionally, self-consistency (Wang et al., 2022) and least-to-most prompting (Zhou et al., 2022)
have further improved the performance of these models on these tasks. However, prompt engineering
is a time-consuming process that requires a lot of trial and error, and it is still difficult for LLMs to
consistently perform well or achieve satisfactory results in solving mathematical problems. Moreover,
simply scaling the data and model size is not an efficient way to improve a model’s mathematical
reasoning abilities. Instead, pretraining on math-related corpora has been shown to consistently
enhance these capabilities (Hendrycks et al., 2021; Lewkowycz et al., 2022; Taylor et al., 2022;
Lightman et al., 2023). Additionally, fine-tuning on math-related instruction-following datasets (Si
et al., 2023; Yuan et al., 2023a; Luo et al., 2023a; Yue et al., 2023; Chern et al., 2023a; Yu et al.,
2023), has also been effective and more cost-effective than math-specific pretraining. Despite their
limitations in terms of accuracy, LLMs still have significant potential to assist users with practical
mathematical problems. There is ample scope for further development in this area.
7 CONCLUSION
In this report, we present the QWEN series of large language models, which showcase the latest
advancements in natural language processing. With 14B, 7B, and 1.8B parameters, these models
have been pre-trained on massive amounts of data, including trillions of tokens, and fine-tuned using
cutting-edge techniques such as SFT and RLHF. Additionally, the QWEN series includes specialized
models for coding and mathematics, such as CODE-QWEN, CODE-QWEN-CHAT, and MATH-QWENCHAT, which have been trained on domain-specific data to excel in their respective fields. Our results
demonstrate that the QWEN series is competitive with existing open-source models and even matches
the performance of some proprietary models on comprehensive benchmarks and human evaluation.
We believe that the open access of QWEN will foster collaboration and innovation within the
community, enabling researchers and developers to build upon our work and push the boundaries of
what is possible with language models. By providing these models to the public, we hope to inspire
new research and applications that will further advance the field and contribute to our understanding of
the variables and techniques introduced in realistic settings. In a nutshell, the QWEN series represents
a major milestone in our development of large language models, and we are excited to see how it will
be used to drive progress and innovation in the years to come.
"""

In [919]:
"""
split chunks
"""

text = ori_text#.replace('\n', '')
matches = re.finditer(r'\n', text)
print("text length:", len(text))

max_chunk_length = 2048
result_chunks = []

left = 0
right = 0
match = -1
while(match is not None):
    try:
        match = next(matches)
        midx = match.start()
    except:
        match = None
        midx = len(text)
    
    if midx - left < max_chunk_length:
        right = midx
    else:
        # text split is longer than max_chuck_length
        if left >= right:
            while(midx - left >= max_chunk_length):
                chunk = text[left:left+max_chunk_length]
                left += max_chunk_length
                right += max_chunk_length
                result_chunks.append(chunk)
            right = midx
        chunk = text[left:right+1]
        left = right+1
        right = midx
        result_chunks.append(chunk)
        
if left < right: # get last split
    chunk = text[left:right+1]
    result_chunks.append(chunk)
    
print("num chunk:", len(result_chunks))

text length: 21249
num chunk: 11


In [920]:
%%time
"""
extract ToC
"""

api_server_url = "http://localhost:21122"
instruct = (
    "You are a Table of Contents extractor. "
    "User will speak to you questions. "
    "You must reply only with [목차(Table of Contents)] part extracted from the questions. "
    "You must keep original text. Do not change original text. "
    "And you must not involve [dotted line, page number, 제목(title), 참고문헌(reference), 부록(appendix), content, explanation, summary, predicted]. "
    "When there is no Table of Contents, you must reply with \"없음\". "
    "do not write explanations."
)
prompt_template = """### System:
{instruct}
### User: {data}

### Assistant:"""
def send_extract_toc(tocs):
    global idx
    # for _ in range(2):
    while(1):
        if idx > len_data - 1:
            break
        lock.acquire()
        pidx = idx
        data = result_chunks[pidx]
        idx += 1
        lock.release()
        
        print(f"{idx}/{len_data}", '\t\t\t\t\t\t', end='\r')#
        
        # response
        prompt = prompt_template.format(
            instruct=instruct,
            data=data
        )
        input_json = {
            "model_name": "MDIEM-toc3", #"OLLM-Small_2024.01.16", #"MDIEM-toc3",
            "prompt": prompt,
            "temperature": 0.7,
            "top_p": 0.8,
            "max_new_tokens": 512,
        }

        ret = requests.post(
            api_server_url + "/worker_generate",
            json=input_json,
            timeout=30,
        )

        result = ret.json()['text'][len(input_json['prompt'])+1:]
        tocs[pidx] = result

tocs = [""] * len(result_chunks)
threads = []
idx = 0
lock = threading.Lock()
len_data = len(result_chunks)
n_thread = 16

for i in range(n_thread):
    t = threading.Thread(target=send_extract_toc, args=(tocs,)) # 
    t.start()
    threads.append(t)
    
for t in threads:
    t.join()
    
    
first_toc = '\n'.join(tocs)
# response
prompt = prompt_template.format(
    instruct=instruct,
    data=first_toc
)
input_json = {
    "model_name": "MDIEM-toc3", #"OLLM-Small_2024.01.16", # 
    "prompt": prompt,
    "temperature": 0.7,
    "top_p": 0.8,
    "max_new_tokens": 1024,
}

ret = requests.post(
    api_server_url + "/worker_generate",
    json=input_json,
    timeout=30,
)

second_toc = ret.json()['text'][len(input_json['prompt'])+1:]
print(second_toc)

1 Introduction
2 Architectural details
2.1 Sparse Mixture of Experts
3 Results
3.1 Multilingual benchmarks
3.2 Long range performance
3.3 Bias Benchmarks

CPU times: user 50.8 ms, sys: 14.6 ms, total: 65.4 ms
Wall time: 33 s


In [921]:
print(first_toc)

1 Introduction

2 Architectural details
2.1 Sparse Mixture of Experts

없음

3 Results

없음

3.1 Multilingual benchmarks
Table 3: Comparison of Mixtral with Llama 2 70B and GPT-3.5. Mixtral outperforms or matches Llama 2 70B and GPT-3.5 performance on most metrics.
Active
Params
French German Spanish Italian
Model arc-c HellaS MMLUarc-c HellaS MMLUarc-c HellaS MMLUarc-c HellaS MMLU
LLaMA 1 33B 33B 39.3% 68.1% 49.9% 41.1% 63.3% 48.7% 45.7% 69.8% 52.3% 42.9% 65.4% 49.0%
LLaMA 2 70B 70B 49.9% 72.5% 64.3% 47.3% 68.7% 64.2% 50.5% 74.5% 66.0% 49.4% 70.9% 65.1%
Mixtral 8x7B 13B 58.2% 77.4% 70.9% 54.3% 73.0% 71.5% 55.4% 77.6% 72.5% 52.8% 75.1% 70.9%

3.2 Long range performance
3.3 Bias Benchmarks

없음

없음

없음

없음



In [922]:
"""
make ToC-Contents Groups
"""

toc_string = re.sub(r'\n+', '\n', second_toc)
parts = toc_string.split('\n')

MIN_SPLIT_LEN = 1

def remove_empty(parts):
    new_parts = []
    for part in parts:
        if len(part) <= MIN_SPLIT_LEN: continue
        new_parts.append(part)
    return new_parts

parts = remove_empty(parts)

parts_indexs = []
for part in parts:
    matched = [match.start() for match in re.finditer(part, ori_text)]
    for match in matched:
        parts_indexs.append((part, match))
    print(part, matched)
    
parts_indexs = sorted(parts_indexs, key=lambda x: x[1])
print(parts_indexs)

toc_dict = {}
for pdata1, pdata2 in zip(parts_indexs, parts_indexs[1:] + [('', len(ori_text)-1)]):
    name, left = pdata1
    left += len(name)
    _, right = pdata2
    
    if right - left > MIN_SPLIT_LEN:
        if name not in toc_dict:
            toc_dict[name] = [ori_text[left:right]]
        else:
            toc_dict[name].append(ori_text[left:right])

1 Introduction [0]
2 Architectural details [2636]
2.1 Sparse Mixture of Experts [3189]
3 Results [6518]
3.1 Multilingual benchmarks [11273]
3.2 Long range performance [12235]
3.3 Bias Benchmarks [13127]
[('1 Introduction', 0), ('2 Architectural details', 2636), ('2.1 Sparse Mixture of Experts', 3189), ('3 Results', 6518), ('3.1 Multilingual benchmarks', 11273), ('3.2 Long range performance', 12235), ('3.3 Bias Benchmarks', 13127)]


In [925]:
%%time
"""
Summary ToC-Contents Groups
"""

MIN_CONTENT_LEN = 10

api_server_url = "http://localhost:21122"
# I want you to act as a summaryist. I will give you sentences. "
# f"I want you to only reply with a summarization based on the sentences. do not write explanations.
# "do not write explanations. summarize the sentences as briefly as possible: {query}"
instruct = (
    "do not write explanations. 다음 문장을 한글로 번역하고 최대한 짧게 요약해주세요: {query}"
)
prompt_template = """### System:
This is a system prompt, please behave and help the user.

### User: {instruct}

### Assistant:"""
def send_summary(summaries):
    global idx
    # for _ in range(2):
    while(1):
        if idx > len_data - 1:
            break
        lock.acquire()
        pidx = idx
        data = toc_dict.get(parts[pidx])
        idx += 1
        lock.release()
        
        print(f"{idx}/{len_data}", '\t\t\t\t\t\t', end='\r')#
        
        if data is None:
            continue
        data = '\n'.join(data)
        if len(data) < MIN_CONTENT_LEN:
            # summaries[pidx] = data
            continue
        
        # response
        prompt = prompt_template.format(
            instruct=instruct.format(query=data),
        )
        input_json = {
            "model_name": "OLLM-Large_2023.11.20",# "MDIEM-toc3",
            "prompt": prompt,
            "temperature": 0.8,
            "top_p": 0.7,
            "max_new_tokens": 1024,
        }

        ret = requests.post(
            api_server_url + "/worker_generate",
            json=input_json,
            timeout=300,
        )

        result = ret.json()['text'][len(input_json['prompt'])+1:]
        summaries[pidx] = result

summaries = [""] * len(parts)
threads = []
idx = 0
lock = threading.Lock()
len_data = len(parts)
n_thread = 16

for i in range(n_thread):
    t = threading.Thread(target=send_summary, args=(summaries,)) # 
    t.start()
    threads.append(t)
    
for t in threads:
    t.join()

CPU times: user 46.5 ms, sys: 14.2 ms, total: 60.6 ms
Wall time: 1min 27s


In [926]:
for idx in range(len(parts)):
    print(parts[idx], '\n', summaries[idx], '\n')

1 Introduction 
 이 논문에서는 Mixtral 8x7B라는 오픈 가중치가 있는 스페어스 믹스추러 모델(SMoE)을 소개합니다. 이 모델은 Apache 2.0 라이선스로 라디오 2 70B와 GPT-3.5를 대부분의 벤치마크에서 능가합니다. 토큰당 하위 집합을 사용하여 추론 속도를 낮은 배치 크기에서 빠르게 하고 대용량 배치 크기에서 처리량을 높입니다. Mixtral은 또한 32K 토큰의 맥락 크기를 사용하여 다국어 데이터로 사전 훈련되었습니다. 이 모델은 수학, 코드 생성 및 다국어 이해가 필요한 작업에서 Llama 2 70B를 크게 앞섭니다. 또한 Mixtral 8x7B-Instruct라는 명령어를 따르는 대화형 모델도 소개되며, 이 모델은 인간 평가 벤치마크에서 GPT-3.5 Turbo, Claude-2.1, Gemini Pro, Llama 2 70B-chat 모델을 크게 앞섭니다. Mixtral 8x7B와 Mixtral 8x7B-Instruct는 모두 Apache 2.0 라이선스로 학술 및 상업 목적으로 무료로 공개되어 다양한 용도로 사용할 수 있습니다. 

2 Architectural details 
 
번역결과번역결과 
매개변수 값
출력 Unit 4096
층 수 32
헤드 출력 크기 128
은닉 출력 크기 14336
헤드 수 32
키-관련 헤드 수 8
컨텍스트 길이 32768
사전 크기 32000
전문가 수 8
상위 k 전문가 2
표 1: 모델 아키텍처.
Mixtral은 변환기 아키텍처[31]을 기반으로하고 동일한 수정 사항을 사용합니다[18], 완전한 밀접한 컨텍스트 길이는 32k 토큰으로 지원되며, 피드 포워드 블록이 전문가 층(섹션 2.1)으로 대체됩니다.
표 1에서 모델 아키텍처 매개 변수를 요약합니다.
설명이 없습니다. 

2.1 Sparse Mixture of Experts 
 다음 문장은 Mixture of Experts 레이어(Figure 1)에 대한 간략한 개요를 제공합니다. 자세한 내용은 [12]를 참조하세요. 이 모듈은 

In [ ]:
for part in parts:
    if part in toc_dict:
        print(part, '\n', len(toc_dict[part]), '\n', toc_dict[part], '\n')

In [890]:
640/360

1.7777777777777777

In [891]:
2688/1520

1.768421052631579

In [892]:
import cv2

In [893]:
img = cv2.imread("GM2-RAMP2-ENT-2024-1-07--09-15-30-entrycar.jpg")
resized = cv2.resize(img, (640, 360))
cv2.imwrite("GM2-RAMP2-ENT-2024-1-07--09-15-30-entrycar-resized.jpg", resized)

True

In [915]:
class A:
    @classmethod
    def abc(cls):
        print("abc", cls.count)

In [917]:
A.abc()

AttributeError: type object 'A' has no attribute 'count'

In [910]:
a = A()

In [916]:
a.abc()

abc
